# Parsing the output from dbCAN

The CAZyme prediction tools write out their output files in different formats, and do not present the data in a standardised manner across all tools. Additionally, the output from the prediction tools does not enable easy parsing of the data by the [Sklearn](https://scikit-learn.org/stable/index.html) library, which performs the statistical analysis of the tools preformances.

This notebook explains the theorlogy and method used by the `pyrewton.cazymes.prediction.parse` submodule to parse the output from the CAZyme prediction tool dbCAN.

[dbCAN](https://pubmed.ncbi.nlm.nih.gov/29771380/) is a CAZyme prediction tool that incorporates 3 protein function prediction tools:
- [DIAMOND](https://pubmed.ncbi.nlm.nih.gov/25402007/) - predicts protein function based on sequence identitiy, and is very similar in approach to BLAST.
- [HMMER](https://pubmed.ncbi.nlm.nih.gov/29905871/) - predicts protein function based upon sequence identity measured by comparing hidden Markov models (HMMs)
- [Hotpep](https://link.springer.com/article/10.1186/s12859-017-1625-9) - k-mer based protein function prediction

The prediction outputs from DIAMOND, HMMER and Hotpep are separated and the consensus result (results that at least two of the results agree on) are collected and defined as the dbCAN consensus result (as is the practise of the dbCAN developers).

### Notebook structure

The section titled 'exploring the output' shows the exploration of the tools output files. This is to demonstrate how the data is constructed and presented, and forms a foundation for explaining the developed method of parsing the data.

The section 'parsing the output' shows the method of parsing the data from the CAZyme prediction tool.

The final section shows any additional parsing that was performed to make the statistical analysis of the tool's performance easier. This typically includes removing duplicates, and combining rows that represent data for the same predicted CAZyme.


## Contents

- [Imports](#notebook_imports)
- [Exploring the output file of dbCAN](#outputfiles)
    - [Overview.txt output file](#exp_over)
    - [HMMER.out output file](#exp_hmmer)
    - [DIAMOND.out output file](#exp_diamond)
    - [Hotpep.out output file](#exp_hotpep)
    - [uniIinput output file](#exp_uniinput)
- [Choosing the dbCAN output data and files to parse](#choice)
_______________________________________________________________________________________________
- [Parsing the overview.txt file](#par_over)
- [Defining the data model](#model)
    - [HMMER](#hmmer)
        - [Parsing HMMER data from overview.txt](#over_hmmer)
    - [Hotpep](#hotpep)
        - [Parsing Hotpep data from overview.txt](#over_hotpep)
        - [Parsing the Hotpep.out file](#hotpep_out)
    - [DIAMOND](#diamond)
        - [Parsing DIAMOND data from overview.txt](#over_diamond)
    - [dbCAN (consensus result)](#dbcan)
        - [Retrieving the HMMER, Hotpep and DIAMOND data for each line in the overview.txt file](#retrieve_hmmer_hotpep_diamond)
        - [Getting the dbCAN consensus](#dbcan_con)
    - [Retrieve non-CAZymes](#non-caz)
- [The Final Functions](#final_fun)
        

<a id="notebook_imports"></a>

## Imports

In [2]:
# Install necessary libraries
!pip3 install pandas
!pip3 install numpy

import re

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

<a id="outputfiles"></a>

## Exploring the output files of dbCAN

dbCAN produces several output files. This section of the notebook explores what data is included in each, what data we want to retrieve and explain which file were selected for retrieving data.

The output files produced by dbCAN are:
- overview.txt
- diamond.out
- hmmer.out
- Hotpep.out

<a id="exp_over"></a>

### Overview.txt output file

As its name infers, the overview.txt file summarises the predictions from HMMER, DIAMOND and Hotpep. Each row contains a unique protein and contains for the following information:
- "HMMER" - predicted CAZy (sub)families and amino acid domain range
- "DIAMOND" - predicted CAZy (sub)families
- "Hotpep" - predicted CAZy(sub)families and the number of the associated k-mer group


In [3]:
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()
    
for line in overview_file[:5]:
    print(line)

Gene ID	HMMER	Hotpep	DIAMOND	#ofTools
ATY58323.1	CE5(83-249)	-	CE5	2
ATY58330.1	AA11(50-241)	AA11(2)	AA11	3
ATY58395.1	AA7(55-488)	-	-	1
ATY58396.1	AA6(3-198)	AA6(1)	-	2


Lets check how the data is separated.

In [4]:
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()
    
for line in overview_file[:5]:
    print(repr(line))

'Gene ID\tHMMER\tHotpep\tDIAMOND\t#ofTools'
'ATY58323.1\tCE5(83-249)\t-\tCE5\t2'
'ATY58330.1\tAA11(50-241)\tAA11(2)\tAA11\t3'
'ATY58395.1\tAA7(55-488)\t-\t-\t1'
'ATY58396.1\tAA6(3-198)\tAA6(1)\t-\t2'


We can see here that the data is separated by tabulated white space, which makes it easy for to parse the data into a simple dataframe to help us read the data that is containined in the overview.txt file

In [5]:
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()

df_data = []

for line in overview_file[:20]:
    line = line.split("\t")
    df_data.append(line)

df = pd.DataFrame(df_data, columns=["Gene ID","HMMER","Hotpep","DIAMOND","#ofTools"])
df

,Gene ID,HMMER,Hotpep,DIAMOND,#ofTools
0,Gene ID,HMMER,Hotpep,DIAMOND,#ofTools
1,ATY58323.1,CE5(83-249),-,CE5,2
2,ATY58330.1,AA11(50-241),AA11(2),AA11,3
3,ATY58395.1,AA7(55-488),-,-,1
4,ATY58396.1,AA6(3-198),AA6(1),-,2
5,ATY58475.1,AA3_2(28-645),AA3(6),AA3,3
6,ATY58478.1,GH16(79-222),GH16(14),GH16,3
7,ATY58486.1,GH18(30-280),GH18(92)+CBM1(13)+CBM19(1),CBM1+GH18,3
8,ATY58494.1,GH5_24(160-472),GH5(55),GH5_24,3
9,ATY58512.1,GH55(20-743),GH55(5),GH55,3


From this we can see more clearly what data is included in the overview.txt file, and shows what was said before:
- "HMMER" - predicted CAZy (sub)families and amino acid domain range
- "DIAMOND" - predicted CAZy (sub)families
- "Hotpep" - predicted CAZy(sub)families and the number of the associated k-mer group

The '#ofTools' lists the number of tools (out of HMMER, DIAMOND and Hotpep) say a protein is a CAZyme.

The last check is to see if a unique protein is contained per line, and we can see below that a unique protein is contained in each row.

In [6]:
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()

df_data = []

for line in overview_file[:20]:
    line = line.split("\t")
    df_data.append(line)

df = pd.DataFrame(df_data, columns=["Gene ID","HMMER","Hotpep","DIAMOND","#ofTools"])

# check for duplicates
dup_rows = df.duplicated(keep=False, subset="Gene ID")
df[dup_rows]

,Gene ID,HMMER,Hotpep,DIAMOND,#ofTools


<a id="exp_hmmer"></a>

### HMMER.out output file

The HMMER output file contains more information about the data producsed by HMMER during the analysis of the input FASTA file containing protein sequences.

First, lets take a look at the data and how it is separated.

In [7]:
with open ("hmmer.out", "r") as ofh:
    hmmer_file = ofh.read().splitlines()
    
for line in hmmer_file[:5]:
    print(repr(line))

'HMM Profile\tProfile Length\tGene ID\tGene Length\tE Value\tProfile Start\tProfile End\tGene Start\tGene End\tCoverage'
'CE5.hmm\t189\tATY58323.1\t250\t1.2e-42\t2\t188\t83\t249\t0.9841269841269841'
'AA11.hmm\t191\tATY58330.1\t441\t2.6e-68\t1\t188\t50\t241\t0.9790575916230366'
'AA7.hmm\t458\tATY58395.1\t499\t3e-47\t8\t453\t55\t488\t0.9716157205240175'
'AA6.hmm\t195\tATY58396.1\t201\t3.9e-81\t1\t193\t3\t198\t0.9846153846153847'


The data is separated by tabulated white space ('\t'), which means the data can be parsed and produce a dataframe again to help make it clearer what data is in the hmmer.out file.

In [8]:
with open ("hmmer.out", "r") as ofh:
    hmmer_file = ofh.read().splitlines()

df_data = []
column_names = hmmer_file[0].split("\t")

for line in hmmer_file[1:20]:
    line = line.split("\t")
    df_data.append(line)

df = pd.DataFrame(df_data, columns=column_names)
df

,HMM Profile,Profile Length,Gene ID,Gene Length,E Value,Profile Start,Profile End,Gene Start,Gene End,Coverage
0,CE5.hmm,189,ATY58323.1,250,1.2e-42,2,188,83,249,0.9841269841269841
1,AA11.hmm,191,ATY58330.1,441,2.6e-68,1,188,50,241,0.9790575916230366
2,AA7.hmm,458,ATY58395.1,499,3e-47,8,453,55,488,0.9716157205240175
3,AA6.hmm,195,ATY58396.1,201,3.9e-81,1,193,3,198,0.9846153846153847
4,AA3_2.hmm,568,ATY58475.1,649,1.5e-96,2,567,28,645,0.9947183098591549
5,GH16.hmm,189,ATY58478.1,556,5.6e-25,52,176,79,222,0.656084656084656
6,GH18.hmm,296,ATY58486.1,406,9.8e-27,3,201,30,280,0.668918918918919
7,GH5_24.hmm,315,ATY58494.1,505,9.3e-148,1,312,160,472,0.9873015873015873
8,GH55.hmm,740,ATY58512.1,745,5.2e-205,11,739,20,743,0.9837837837837838
9,GH16.hmm,189,ATY58528.1,364,6.7e-21,10,189,62,283,0.9470899470899471


Next, we can check that a unique protein is contained in each row by checking for duplicates.

In [9]:
with open ("hmmer.out", "r") as ofh:
    hmmer_file = ofh.read().splitlines()

df_data = []
column_names = hmmer_file[0].split("\t")

for line in hmmer_file:
    line = line.split("\t")
    df_data.append(line)

df = pd.DataFrame(df_data, columns=column_names)
print("number of proteins in hmmer.out =", len(df["Gene ID"]))

# check for duplicates
dup_rows = df.duplicated(keep=False, subset="Gene ID")
df[dup_rows]

number of proteins in hmmer.out = 337


,HMM Profile,Profile Length,Gene ID,Gene Length,E Value,Profile Start,Profile End,Gene Start,Gene End,Coverage
35,GH18.hmm,296,ATY59188.1,1774,1.9e-53,5,282,518,863,0.9358108108108109
36,CBM66.hmm,155,ATY59188.1,1774,1.5e-27,2,155,1301,1461,0.9870967741935484
49,GH54.hmm,316,ATY59698.1,496,5.9e-131,1,315,25,333,0.9936708860759493
50,CBM42.hmm,136,ATY59698.1,496,5e-50,2,135,351,490,0.9779411764705882
57,GH43_34.hmm,280,ATY59890.1,514,1.9e-97,1,280,55,335,0.9964285714285714
58,CBM66.hmm,155,ATY59890.1,514,2.8e-23,2,154,351,508,0.9806451612903225
77,CBM66.hmm,155,ATY60367.1,555,2.1e-19,4,150,219,369,0.9419354838709677
78,CBM66.hmm,155,ATY60367.1,555,4.7e-25,3,152,388,547,0.9612903225806452
82,GT2_Chitin_synth_1.hmm,163,ATY60416.1,923,6.6e-71,1,163,230,398,0.9938650306748467
83,GT2_Chitin_synth_2.hmm,527,ATY60416.1,923,1.1e-22,203,404,375,584,0.38140417457305503


From this we can see that instead of containing a unique protein per row, HMMER lists all the CAZy families predicted for a protein on separate lines.

**What data is stored in hmmer.out**
The information defining the fields in the HMMER output file were taken from the HMMER [documentation](http://eddylab.org/software/hmmer/Userguide.pdf) and [Read The Docs](https://hmmer-web-docs.readthedocs.io/en/latest/result.html?highlight=profile%20start#alignments).

**HMM Profile**
Is the name of the HMM profile that models the CAZy family HMMER has predicted CAZy would catalogue the query protein under.

**Profile length**
This has been interpretted as the length in amino acids of the HMM profile listed under 'HMM Profile'.

**Gene ID**
This is the accession of the query protein represented in the line of the hmmer.out file.

**E Value**
This is the E value score of the alignemnt, this how HMMER ranks the matches between query proteins and HMM models.

**Profile Start/End and Gene Start/End**
These have been interrepreted to be in relation to what the HMMER documentation lists as the 'Query start/end': "The start/end of the MEA alignment of this domain/hit with respect to the profile HMM, which directly relates to the query sequence for phmmer. For hmmsearch, the number corresponds to the match states that HMMER determined from the initial input alignment." - from [ReadTheDocs](https://hmmer-web-docs.readthedocs.io/en/latest/result.html?highlight=profile%20start#alignments)

The 'Gene Start' and 'Gene End' match the amino acid domain ranges that are listed in the overview.txt file.

**Coverage**
"The fraction of total residues that are shared is reported as the coverage in theesl-alimapoutput." - from [documentation](http://eddylab.org/software/hmmer/Userguide.pdf)

**What data is needed from the hmmer.out file**

For the evaluation performed by `pyrewton` only the predicted CAZy families are needed. For down stream analysis, the predicted domain ranges may also be informative. `pyrewton` uses the E value and coverage cut offs determined by dbCAN, and thus are not needed for the evaluation of the performance of dbCAN by `pyrewton`, although some users may find the data informative. Therefore, they are able to find the data in hmmer.out, but the only data wanted from HMMER is the predicted CAZy (sub)families and their associated domain range.

<a id="exp_diamond"></a>

### diamond.out output file

The DIAMOND output file contains more information about the data produced by DIAMOND during the analysis of the input FASTA file containing protein sequences.

Let's take an inital look at the data and check how the data is separated.

In [10]:
with open ("diamond.out", "r") as ofh:
    diamond_file = ofh.read().splitlines()
    
for line in diamond_file[:5]:
    print(repr(line))

'Gene ID\tCAZy ID\t% Identical\tLength\tMismatches\tGap Open\tGene Start\tGene End\tCAZy Start\tCAZy End\tE Value\tBit Score'
'ATY67280.1\tATY67280.1|GH17|\t100.0\t670\t0\t0\t1\t670\t1\t670\t0.0e+00\t1354.7'
'ATY67467.1\tATY67467.1|GT90|\t100.0\t625\t0\t0\t1\t625\t1\t625\t0.0e+00\t1303.9'
'ATY67289.1\tATY67289.1|CBM48|GH13_8|\t100.0\t691\t0\t0\t1\t691\t1\t691\t0.0e+00\t1440.2'
'ATY67196.1\tATY67196.1|GH18|\t100.0\t365\t0\t0\t1\t365\t1\t365\t4.6e-211\t736.5'


Again the data is separated by tabulated white space ('\t'), which means the data can be parsed and produce a dataframe again to help make it clearer to see what data is included in the diamond.out file. The names of the columns are stored in the first line of the file.

In [11]:
with open ("diamond.out", "r") as ofh:
    diamond_file = ofh.read().splitlines()

df_data = []
column_names = diamond_file[0].split("\t")

for line in diamond_file[1:20]:
    line = line.split("\t")
    df_data.append(line)

df = pd.DataFrame(df_data, columns=column_names)
df

,Gene ID,CAZy ID,% Identical,Length,Mismatches,Gap Open,Gene Start,Gene End,CAZy Start,CAZy End,E Value,Bit Score
0,ATY67280.1,ATY67280.1|GH17|,100.0,670,0,0,1,670,1,670,0.0e+00,1354.7
1,ATY67467.1,ATY67467.1|GT90|,100.0,625,0,0,1,625,1,625,0.0e+00,1303.9
2,ATY67289.1,ATY67289.1|CBM48|GH13_8|,100.0,691,0,0,1,691,1,691,0.0e+00,1440.2
3,ATY67196.1,ATY67196.1|GH18|,100.0,365,0,0,1,365,1,365,4.6e-211,736.5
4,ATY67205.1,EAA64118.1|CBM20|GH15|,46.4,539,263,8,18,541,707,1234,3.0e-140,501.9
5,ATY67226.1,ATY67226.1|GH18|,100.0,482,0,0,1,482,1,482,1.5e-286,987.6
6,ATY67472.1,ATY67472.1|AA3_2|,100.0,587,0,0,1,587,1,587,0.0e+00,1204.1
7,ATY67423.1,AXG41146.1|GH6|,37.6,710,414,10,179,882,180,866,3.1e-118,429.5
8,ATY67327.1,ATY67327.1|GT32|,100.0,396,0,0,1,396,1,396,1.9e-226,787.7
9,ATY67309.1,ATY67309.1|GT20|,100.0,602,0,0,1,602,1,602,0.0e+00,1220.3


We can check if each row contains a unique protein, by parsing the complete diamond.out file and check for duplicates.

In [12]:
with open ("diamond.out", "r") as ofh:
    diamond_file = ofh.read().splitlines()

df_data = []
column_names = diamond_file[0].split("\t")

for line in diamond_file:
    line = line.split("\t")
    df_data.append(line)

df = pd.DataFrame(df_data, columns=column_names)

# check for duplicates
dup_rows = df.duplicated(keep=False, subset="Gene ID")
df[dup_rows]

,Gene ID,CAZy ID,% Identical,Length,Mismatches,Gap Open,Gene Start,Gene End,CAZy Start,CAZy End,E Value,Bit Score


The above shows there is a unique protein per row, thus all CAZy families listed in the 'CAZy ID' column contains all CAZy families predicted to be in a CAZyme. Multiple CAZy families predicted for a CAZyme indicates multiple CAZyme domains are predicted to be present in a protein.

**What data is stored in diamond.out**
The explanations for the data in the diamond.out is primarly taken from the [DIAMOND documentation](https://github.com/bbuchfink/diamond/wiki/1.-Tutorial). Quotes are directlt taken from the DIAMOND documentation.

**Gene ID**
This has been interpretted to be the column that DIAMOND lists as the 'Query accession' in its documentation. "The accession of the sequence that was the search query against the database, as specified in the input FASTA file after the > character until the first blank."

**Sequence identity**
"The percentage of identical amino acid residues that were aligned against each other in the local alignment."

**Length**
"The total length of the local alignment, which including matching and mismatching positions of query and subject, as well as gap positions in the query and subject."

**Mismatches**
"The number of non-identical amino acid residues aligned against each other."

**Gap Open**
This has been interpretted to be what DIAMOND lists as 'Gene Openings' in its documentation. "The number of gap openings."

**Gene Start**
This has been interpretted to be what DIAMOND lists as the 'Query start' in its documentation. "The starting coordinate of the local alignment in the query (1-based)."

**Gene End**
This has be interpretted to be what DIAMOND lists as the 'Gene end' in its documentation. "The ending coordinate of the local alignment in the query (1-based)."

**CAZy Start**
This has been interpretted to be what DIAMOND list as the 'Target start' in the documentation. "The starting coordinate of the local alignment in the target (1-based)."

**CAZy End**
This has been interpretted to be what DIAMOND lists as 'Target end' in its documentation. "The ending coordinate of the local alignment in the target (1-based)."

**E-value**
"The expected value of the hit quantifies the number of alignments of similar or better quality that you expect to find searching this query against a database of random sequences the same size as the actual target database. This number is most useful for measuring the significance of a hit. By default, DIAMOND will report all alignments with e-value < 0.001, meaning that a hit of this quality will be found by chance on average once per 1,000 queries."

**Bit score**
"The bit score is a scoring matrix independent measure of the (local) similarity of the two aligned sequences, with higher numbers meaning more similar. It is always >= 0 for local Smith Waterman alignments."

**What data do we want?**
The 'Length' and 'Start/End' columns included the length of the entire protein, not predictions of the CAZyme domain locations, therefore, their data is not informative. The remaining columns, apart from the 'CAZy ID' provide information on the score of the alignment, which is not informative for the evaluation `pyrewton` performs. `pyrewton` uses the default E-value/bit-score cut offs. This may be informative data depending on the personal needs of the user, but becuase it is not needed by `pyrewton` it is not collected. The only needed data is the predicted CAZy families.

<a id="exp_hotpep"></a>

### Hotpep.out output file

The Hotpep output file contains more information about the data produced by Hotpep, from the analysis of the input FASTA file containing protein sequences.

Let's take an inital look at the data and check how the data is separated.

In [13]:
with open ("Hotpep.out", "r") as ofh:
    hotpep_file = ofh.read().splitlines()
    
for line in hotpep_file[:5]:
    print(repr(line))

'CAZy Family\tPPR Subfamily\tGene ID\tFrequency\tHits\tSignature Peptides\tEC number'
'CE16\t2\tATY65081.1\t6.91\t12\tTGGRRF,VTFGDS,IGTNDL,WIGTND,TFGDSY,DSYTDE,FGDSYT,NYAVAG,GDSYTD,LVTFGD,YNYAVA,GTNDLG\t3.1.1.6:22'
'CE3\t10\tATY62702.1\t9.55\t16\tDCIHPT,DYWYDF,SGKTIQ,FAAWSG,WSGKTI,VGDSIT,AGTNDM,GTNDMN,GDSITV,CIHPTN,VLAVDF,AAWSGK,LAVDFT,DSITVG,GDYWYD,AWSGKT\tNA'
'CE4\t143\tATY63454.1\t12.6\t26\tHQVASH,ALTFDD,GECLGD,SHTWSH,MRPPYS,PTYMRP,TVGECL,VGECLG,VTVGEC,GYFPTY,TYMRPP,ECLGDP,YFPTYM,ASHTWS,LGYFPT,HTWSHQ,FPTYMR,FDDGPY,IALTFD,YMRPPY,GHQVAS,LTFDDG,TFDDGP,AKATFF,QVASHT,VASHTW\tNA'
'CE4\t143\tATY64953.1\t7.92\t15\tVTVGEC,ALTFDD,AKATFF,TYMRPP,YMRPPY,EMAFRN,PTYMRP,VGECLG,TVGECL,GECLGD,ECLGDP,LTFDDG,NEMAFR,TFDDGP,KATFFI\tNA'


The individual fields of data are separated by tabulated white space ('\t'), which allows us to separate them out and produce a dataframe to help understand/visualise what data is stored in the Hotpep output file.

In [14]:
with open ("Hotpep.out", "r") as ofh:
    hotpep_file = ofh.read().splitlines()

df_data = []
column_names = hotpep_file[0].split("\t")

for line in hotpep_file[1:20]:
    line = line.split("\t")
    df_data.append(line)

df = pd.DataFrame(df_data, columns=column_names)
df

,CAZy Family,PPR Subfamily,Gene ID,Frequency,Hits,Signature Peptides,EC number
0,CE16,2,ATY65081.1,6.91,12,"TGGRRF,VTFGDS,IGTNDL,WIGTND,TFGDSY,DSYTDE,FGDS...",3.1.1.6:22
1,CE3,10,ATY62702.1,9.55,16,"DCIHPT,DYWYDF,SGKTIQ,FAAWSG,WSGKTI,VGDSIT,AGTN...",NA
2,CE4,143,ATY63454.1,12.6,26,"HQVASH,ALTFDD,GECLGD,SHTWSH,MRPPYS,PTYMRP,TVGE...",NA
3,CE4,143,ATY64953.1,7.92,15,"VTVGEC,ALTFDD,AKATFF,TYMRPP,YMRPPY,EMAFRN,PTYM...",NA
4,CE5,15,ATY65055.1,7.8,12,"LEGYSQ,AVKGVF,GYSQGA,VKGVFL,GLACNV,DAVKGV,AFDA...",3.1.1.74:26
5,CE9,25,ATY66875.1,13.9,36,"MITHLF,KFTNCR,THLFNA,GAESLG,IAGSSI,TMITHL,HHRN...",3.5.1.25:14
6,GH0,2,ATY64043.1,27.4,38,"SNWRGP,TVREDY,LKERLF,VREDYS,GLGASH,ERLFGL,WWVN...",NA
7,GH0,23,ATY60133.1,5.81,6,"GDGVTD,DGVTDD,AKGDGV,GAKGDG,GVTDDT,KGDGVT",NA
8,GH0,68,ATY62132.1,17.9,30,"MVYTYN,RDEMVY,PWRGLG,YKNAIT,GRGRTV,IDWTAA,YNQG...",NA
9,GH0,115,ATY63707.1,9.81,24,"VRAPTT,VANRLT,GDGLKQ,ITQMIR,NPGLMQ,ANRLTG,CVRA...",NA


We can also check if a unique protein is contain a unique protein, but looking for duplicates in the is dataframe. The data below shows that the Hotpep output contains a unique protein per line.

In [15]:
with open ("Hotpep.out", "r") as ofh:
    hotpep_file = ofh.read().splitlines()

df_data = []
column_names = hotpep_file[0].split("\t")

for line in hotpep_file[1:20]:
    line = line.split("\t")
    df_data.append(line)

df = pd.DataFrame(df_data, columns=column_names)

# check for duplicates
dup_rows = df.duplicated(keep=False, subset="Gene ID")
df[dup_rows]

,CAZy Family,PPR Subfamily,Gene ID,Frequency,Hits,Signature Peptides,EC number


**What data is stored in the Hotpep.out file**

**CAZy Family**
This is the predicted CAZy family.

**PPR subfamily**
This was interpretted as the group number of k-mer group for the predicted CAZy family.

**Gene ID**
The accession of the query protein.

**Frequency**
This is potentially the frequency at which CAZyme signature k-mers appear in the query protein.

**Hits**
This is potentially the number of CAZyme signature k-mers found in the protein.

**Signature Peptides**
The k-mers that Hotpep catalogues as signature k-mers of CAZy families and where found in the query protein.

**EC number**
Predicted EC numbers for the query protein.

**What data is wanted from the Hotpep.out file**

For the evaluation of `pyrewton` only the predicted CAZy family. For downstream analysis of the predicted CAZymes, and may be of interest for may researchers are the predicted EC numbers. Therefore, the predicted CAZy families and EC numbers from the Hotpep output are wanted.

<a id="exp_uniinput"></a>

### uniInput output file

The final output file from dbCAN is the file 'uniInput' which is a copy of the FASTA file of protein sequences used as the input for dbCAN.

There is no data in the 'uniInput' output file that is informative of the CAZyme prediction of dbCAN.

<a id="choice"></a>

## Choosing the dbCAN output data and files to parse

The output wanted from dbCAN:

**dbCAN**
- Consensus result (result at that at least two tools agree)

**HMMER**
- Predicted CAZy (sub)families
- Amino acid domain range

**DIAMOND**
- Predicted CAZy (sub)families

**Hotpep**
- Predicted CAZy (sub)families
- EC numbers

All the wanted data can be retrieved from the overview.txt file, except the EC numbers from Hotpep which requires parsing the Hotpep.out file. To minimise the number of files that need to be parsed the wanted data will be retrieved from the overview.txt file, then add in the EC numbers from the Hotpep.out file.

<a id="par_over"></a>

## Parsing the overview.txt file

This section of the notebook covers parsing the overview.txt file. The parsing of each prediction tool (HMMER, Hotpep and DIAMOND) is discussed in the order they appear in the overview.txt file, finishing with finding the consensus dbCAN result.

<a id="model"></a>

### Defining the data model

The explanation of the design of the data model is explained in the note book prediction_data_model.ipynb.

The same data model is used for every CAZyme prediction tool.

For **HMMER** the data we want to retrieve is:
- The protein accession (which is the 'Gene ID')
- The CAZy (sub)families HMMER has predicted CAZy would catalogue the protein under
- Predicted amino acid domain ranges of the predicted CAZy domains (each domain is represented as a CAZy family prediction, thus predicting two CAZy families indicates predicting two CAZyme domains)

For **Hotpep** the data we want to retrieve is:
- The protein accession (which is the 'Gene ID')
- The CAZy (sub)families HMMER has predicted CAZy would catalogue the protein under
- Predicted EC numbers of the predicted CAZy domains (each domain is represented as a CAZy family prediction, thus predicting two CAZy families indicates predicting two CAZyme domains)

The data from **dbCAN and DIAMOND** includes:
- The protein accession
- CAZy family of each domain
- CAZy subfamily of each domain (if not in a subfamily, the subfamily is listed as a null value)

In [16]:
class CazymeDomain:
    """Single CAZyme domain in a protein, predicted by a CAZyme prediction tool.
    
    Each unique predicted CAZy family-subfamily combination by a CAZyme prediction
    tool for a query protein sequence is viewed as a CAZyme domain.]
    
    Every CAZyme domain has a source CAZyme prediction tool that predicted the CAZyme
    domain, a parent CAZyme protein (represented by the protein accession), and CAZy
    family and subfamily combination. If no CAZy subfamily is predicted, the 
    subfamily will be listed as a null value.

    Hotpep, CUPP and eCAMI predict EC numbers for each CAZyme domain. HMMER and CUPP
    predict amino acid domain ranges. Multiple EC numbers and domain ranges can be
    predicted for a single CAZyme domain, therefore, these attritbutes are stored as
    lists.
    
    If an item of data is not or was not predicted by a CAZyme prediction tool, the
    respective data will be stored as a null value.
    """
    
    def __init__(
        self,
        prediction_tool,
        protein_accession,
        cazy_family,
        cazy_subfamily=None,
        ec_numbers=None,
        domain_range=None,
    ):
        """Initiate instance
        
        :attr prediction_tool: str, CAZyme prediciton tool which predicted the domain
        :attr protein_accession: str
        :attr cazy_family: str
        :attr cazy_subfamily: str
        :attr ec_numbers: list (list of str, each str contains a unique EC number)
        :attr domain_range: list (list of str, each str contains a unique domain range)
        """
        self.prediction_tool = prediction_tool
        self.protein_accession = protein_accession
        self.cazy_family = cazy_family
        
        # not all CAZyme domans are catalogued under a CAZy subfamily
        if cazy_subfamily is None:
            self.cazy_subfamiy = np.nan
        else:
            self.cazy_subfamily = cazy_subfamily
        
        # EC numbers are not predicted for all CAZyme domains
        if ec_numbers is None:
            self.ec_numbers = []  # enables adding in EC numbers included in another line of the output file
        else:
            self.ec_numbers = ec_numbers
        
        # Not all prediction tools predict CAZyme domains
        if domain_range is None:
            self.domain_range = []  # enables adding domain range listed in another line of the ouput file
        else:
            self.domain_range = domain_range
    
    def __str__(self):
        return f"-CazymeDomain in {self.protein_accession}, fam={self.cazy_family}, subfam={self.cazy_subfamily}-"
    
    def __repr__(self):
        return f"<CazymeDomain parent={self.protein_accession} fam={self.cazy_family}, subfam={self.cazy_subfamily}>"

    
class CazymeProteinPrediction:
    """Single protein and CAZyme/non-CAZyme prediction by a CAZyme prediction tool"""
    
    def __init__(self, prediction_tool, protein_accession, cazyme_classification, cazyme_domains=None):
        """Initate class instance.
        
        :attr prediction_tool: str, name of CAZyme prediction tool
        :attr protein_accession: str
        :attr cazyme_classification: int, 1=CAZyme, 0=non-CAZyme
        :attr cazyme_domains: list of CazymeDomain instances, or null value if none are given
        """
        self.prediction_tool = prediction_tool
        self.protein_accession = protein_accession
        self.cazyme_classification = cazyme_classification  # CAZyme=1, non-CAZyme=0
        
        # non-CAZymes will have no CAZyme domains
        if cazyme_domains is None:
            self.cazyme_domains = np.nan
        else:
            self.cazyme_domains = cazyme_domains
    
    def __str__(self):
        if self.cazyme_classification == 0:
            return f"-CazymeProteinPrediction, protein={self.protein_accession}, non-CAZyme-"
        else:
            return f"-CazymeProteinPrediction, protein={self.protein_accession}, CAZyme domains={len(self.cazyme_domains)}-"
    
    def __repr__(self):
        return f"<CazymeProteinPrediction, protein={self.protein_accession}>"

    def get_cazyme_classification_dicts(self):
        """Returns a dictionary that contains the protein accession and its CAZyme/non-CAZyme classification.
        
        This dictionary is used to create the cazyme_classification dataframe for evaluating the performance
        of the CAZyme prediction tools in terms of differnetiating between CAZymes (identified with a score of 1)
        and non-CAZymes (identified with a score of 0). The CAZy classes under which the protein is classifed are
        retrieved as well to enable evaluating the CAZyme/non-CAZyme differneitation per CAZy class, and obsever
        differences between this performance between classes. A score of 0 incidates a protein was not classified
        under the class, a score of 1 indicates the protein as classified under the class.
        """
        cazyme_classification_dict = {
            "protein_accession": self.protein_accession,
            "cazyme_classification": self.cazyme_classification,
            "GH": 0,
            "GT": 0,
            "CE": 0,
            "PL": 0,
            "AA": 0,
            "CBM": 0,
        }
        
        # check which CAZy classes the predicted CAZyme domains were classified under
        for domain in cazyme_domains:
            if domain.cazy_family[:3] == "CBM":
                cazyme_classification_dict["CBM"] = 1
            else:
                cazyme_classification_dict[family[:2]] = 1

        return cazyme_classification_dict

    def get_cazy_family_dict(self):
        """Return a dictionary of the predicted CAZy families for a protein.
        
        This dictionary is used to create a dataframe of CAZy family predictions for evaluating the performance
        of the prediction tools to predict the CAZy family of a CAZyme. The dictionary is keyed by the CAZy
        classes, and valued by a list of the corresponding families predicted for the CAZyme. The is to enable
        evaluating the CAZy family prediction performance per class."""
        cazy_family_dict = {
            "GH": [],
            "GT": [],
            "CE": [],
            "PL": [],
            "AA": [],
            "CBM": [],
        }
        
        for domain in self.cazyme_domains:
            if domain.cazy_family[:3] == "CBM":
                cazy_family_dict["CBM"].append(domain.cazy_family)
            else:
                cazy_family_dict[domain.cazy_family[:2].append(domain.cazy_family)]
        
        return cazy_family_dict


<a id="hmmer"></a>

## HMMER

For HMMER the data we want to retrieve is:
- The protein accession (which is the 'Gene ID')
- The CAZy (sub)families HMMER has predicted CAZy would catalogue the protein under
- Predicted amino acid domain ranges of the predicted CAZy domains (each domain is represented as a CAZy family prediction, thus predicting two CAZy families indicates predicting two CAZyme domains)

<a id="over_hmmer"></a>

### Parsing HMMER data from overview.txt

For HMMER the data we want to retrieve is:
- The protein accession (which is the 'Gene ID')
- The CAZy (sub)families HMMER has predicted CAZy would catalogue the protein under
- Predicted amino acid domain ranges of the predicted CAZy domains (each domain is represented as a CAZy family prediction, thus predicting two CAZy families indicates predicting two CAZyme domains)

Lets take a closer look at how the data from HMMER is presented in the overview.txt file.

In [17]:
# Check how text is separated in overview.txt
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()

data_ov = []
for line in overview_file[1:]:  # skip the first line becuase this is the 'column' titles
    line = line.split("\t")
    hmmer = line[1]
    print(hmmer)


CE5(83-249)
AA11(50-241)
AA7(55-488)
AA6(3-198)
AA3_2(28-645)
GH16(79-222)
GH18(30-280)
GH5_24(160-472)
GH55(20-743)
GH16(62-283)
GH64(3-372)
CE5(73-247)
GH76(27-411)
GH38(431-690)
GT2_Glyco_tranf_2_3(265-498)
GH12(501-652)
CE10(413-656)
GH75(8-230)
CE10(428-676)
GH29(148-504)
GT66(23-667)
GH18(42-300)
GH1(124-596)
AA3_1(23-560)
AA3_3(8-607)
CE10(83-315)
CE10(146-417)
GH16(41-301)
CE5(30-241)
GH18(200-546)
GH13_5(37-380)
GH81(318-1002)
GH18(38-369)
GT90(554-821)
GH18(518-863)+CBM66(1301-1461)
GH72(25-328)
GT90(508-783)
GT15(80-356)
AA12(828-1219)
CE10(33-280)
AA3(43-784)
GT21(51-358)
CE10(101-249)
CE4(66-197)
GT2_Glyco_trans_2_3(441-704)
AA1_2(41-367)
GT1(191-513)
GH54(25-333)+CBM42(351-490)
GT4(218-349)
AA7(99-558)
CE10(73-319)
AA7(124-328)
GT1(271-534)
AA3_1(233-765)
GH43_34(55-335)+CBM66(351-508)
AA3_2(40-423)
AA4(66-300)
GT2_Chitin_synth_2(254-541)
GH128(46-257)
GH47(115-578)
GT2_Chitin_synth_2(1205-1712)
GH16(82-216)
GT17(55-346)
GH55(31-773)
GT4(360-514)
AA1_3(48-355)
AA11(21-220

The CAZy familiy is immediately followed by its predicting amino acid domain range in parentheses.

If multple CAZyme domains are predicted (represented by predicting multiple CAZy families), the domains/CAZy families are separated by '+'.

Sometimes proteins predicated as containing GT2 domains are given the CAZy family "GT2_Chitin_synth_2". No reason for this has been found. For this reason and for standardisation, the names of the predicted CAZy families need to be standardised to GT2, and the same performed in case this occurs for other families as well.

Additionally, when no CAZy domains are predicated to be within the protein sequence a result of '-' is provided. It will be better for the final dataframe if this is converted to a true null value.

Lastly, sometimes HMMER predicates the CAZy subfamily. Unlike with eCAMI and CUPP the subfamily is not passed to a separate column, although this is helpful when it comes to evaluating the tools performances. Therefore, when HMMER predicates the CAZy subfamily this result needs to be passed to retrieve the CAZy family and then store the CAZy family and subfamily separately.

If HMMER predicts the protein is not a CAZyme then "-" is the only thing that is written.

HMMER can predict multiple CAZyme domains are in a protein. This is indicated by listing multiple CAZy families for a protein, where each CAZy family presents a different CAZyme domain in the protein. Each domain has a CAZy family, potentially a CAZy subfamily, and amino acid domain range. Each domain is represented by a single HMMERCazymeDomain instance. Each protein is represented by a single HMMERprediction instance, which stores all the CAZyme domains (the HMMERCazymeDomain instances) predicted to be in the protein.

To be able to check if a protein has been parsed from the overview.txt file, the data will be stored in a dictionary. The `overview_dict` will be keyed by protein accessions, and valued by another dictionary, keyed by the name of the CAZyme prediction tools (dbCAN, HMMER, DIAMOND, Hotpep), which is then valued by the respective data model for presenting the prediction tools prediction for the respective protein. To illustrate:  
```
overview_dict = {
    "ATY60367.1": {"HMMER": <HMMERprediction-instance>},
    "ATY60372.1": {"HMMER": <HMMERprediction-instance>},
}
```
This will allow also CAZyme prediction tool data models for each protein to be stored together. To illustrate again:  
```
overview_dict = {
    "ATY60367.1": {
        "HMMER": <HMMERprediction-instance>,
        "DIAMOND": <DIAMONDprediction-instance>,
    },
    "ATY60372.1": {
        "HMMER": <HMMERprediction-instance>,
        "DIAMOND": <DIAMONDprediction-instance>,
    },
}
```

Another thing to note is that sometimes multiple domain ranges are predicted for the same domain. For example:  
`protein=ATY60416.1, fams=['GT2', 'GT2'], subfams=[nan, nan], ranges=['230-398', '375-584']`  
Luckily each line of the overview.txt file contains a unique protein. Therefore, while parsing a single line, when retrieving each predicted CAZyme domain it needs to be checked if the CAZy (sub)family has already been captured. If it has then the additional/alternative predicted amino acid domain range needs to be added to the already captured domain ranges. Consequently, the domain ranges are stored in lists to enable storing all predicted domain ranges for a single CAZyme domain to be stored together.

To keep all predicted data for each CAZyme domain within the protein together, a `HMMERCazymeDomain` instance is created for each predicted CAZyme domain in the protein. The instance includes the CAZy family, subfamily (a null value if not given) and a predicted domain range (a null value if not given, although it always appears to be given).

Multiple domain ranges can be predicted for a single domain, the `HMMERCazymeDomain` instances will be stored in a dictionary keyed by a string containing the CAZy family and subfamily, and valued by the corresponding `HMMERCazymeDomain` instance. After each predicted CAZyme domain is processed this dictionary will be checked to see if the CAZyme domain (identified by its CAZy family and subfamily) has already been catalogued. If it has then the additional domain range prediction will be added to the domain range prediction of the existing `HMMERCazymeDomain` instance. To allow this, the domain range of each CAZyme domain will be stored as a list, and each predicted domain range for the CAZyme domain stored as a separate string in the list.

In [18]:
# Check how text is separated in overview.txt
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()

overview_dict = {}

for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):  # skip the first line becuase this is the 'column' titles
    line = line.split("\t")
    
    protein_accession = line[0]
    
    # check if the protein was classified as a non-CAZyme
    if line[1] == "-":
        cazyme_classification = 0  # non-CAZyme
        
        overview_dict[protein_accession] = {
            "HMMER": CazymeProteinPrediction(
                "HMMER",
                protein_accession,
                cazyme_classification,
            )
        }
        continue
    
    # create CazymeProteinPrediction instance to represent the prediction for the current working protein
    cazyme_classification = 1
    parent_cazyme = CazymeProteinPrediction("HMMER", protein_accession, cazyme_classification)

    # get prediction for HMMER and separate the predicted CAZyme domains
    hmmer_predictions = line[1].split("+")
    
    # create empty dictionary to store CAZyme domains {family: HMMERCazymeDomain instance}
    cazyme_domains = {}
    
    # parse each of the predicted CAZyme domains
    for predicted_domain in hmmer_predictions:

        # separate the CAZy family and amino acid domain range
        predicted_domain = predicted_domain.split("(")
        
        # get the CAZy (sub)family of the predicted domain
        domain_name = predicted_domain[0]
        
        if domain_name.find("_") != -1:  # found in unusal name formating or CAZy subfamily
            try:
                re.match(r"\D{2,3}\d+?_\D", domain_name).group  # check for unusal name formating
                cazy_family = domain_name[:domain_name.find("_")]
                cazy_subfamily = np.nan

            except AttributeError:  # raised if not unusal format but a CAZy subfamily
                cazy_family = domain_name[:domain_name.find("_")]
                cazy_subfamily = domain_name
        
        else:
            cazy_family = domain_name
            cazy_subfamily = np.nan

        # get the predicted amino acid domain range
        domain_range = predicted_domain[1][:-1] # exclude the final ")"
        
        # create identifier for the CAZyme domain to check if it has already been catalogued
        domain_identifier = f"{cazy_family}-{cazy_subfamily}"
        
        # check if the CAZyme domain has already been cataglouged
        try:
            existing_domain = cazyme_domains[domain_identifier]
            
            # CAZyme domain already catalogued, check if the retrieved domain range is the same as in the instance
            if domain_range not in existing_domain.domain_range:
                # append the new domain_range
                existing_domain.domain_range.append(domain_range)
                print("multiple domain ranges=", existing_domain, existing_domain.domain_range, protein_accession)
        
        except KeyError:  # raised if the CAZyme domain has not yet been catalogued
            cazyme_domains[domain_identifier] = CazymeDomain(
                prediction_tool="HMMER",
                protein_accession=protein_accession,
                cazy_family=cazy_family,
                cazy_subfamily=cazy_subfamily,
                domain_range=[domain_range],
            )
    
    # add CAZyme domains to the parent CAZyme
    parent_cazyme.cazyme_domains = list(cazyme_domains.values())
    
    # add the protein to the dictionary of all parsed proteins
    try:
        overview_dict[protein_accession]
        
        try:
            overview_dict[protein_accession]["HMMER"]
            print("WARNING -- PROTEIN ALREADY CATALOGUED WITH HMMER PREDICTION")
        
        except KeyError:  # raised it not HMMER prediction catagloued yet
            overview_dict[protein_accession]["HMMER"] = parent_cazyme
    
    except KeyError:  # raised if not protein yet catalogued
        overview_dict[protein_accession] = {"HMMER": parent_cazyme}
        
    print(parent_cazyme)

print("number of parsed proteins=", len(list(overview_dict.values())))

-CazymeProteinPrediction, protein=ATY58323.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58330.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58395.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58396.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58475.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58478.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58486.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58494.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58512.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58528.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58531.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58532.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58571.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58579.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58636.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY586

From prining the class representative strings we can see that the multiple CAZyme domains are stored under a parent CAZyme (for example see protein ATY59188.1).

By comparing the number of protein accessions listed in the `overview_dict` and the number of proteins in the overview.txt file, we can see that all proteins were parsed.

To improve the robustness of the method we can add 'quality' checks to check the expected data is retrieved. This is achieved by using regular expressions, for example, to ensure a believed CAZy subfamily is in the correct format for a CAZy subfamily.

In [19]:
# Check how text is separated in overview.txt
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()

overview_dict = {}

for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):  # skip the first line becuase this is the 'column' titles
    line = line.split("\t")
    
    protein_accession = line[0]
    
    # check if the protein was classified as a non-CAZyme
    if line[1] == "-":
        cazyme_classification = 0  # non-CAZyme
        
        overview_dict[protein_accession] = {
            "HMMER": CazymeProteinPrediction(
                "HMMER",
                protein_accession,
                cazyme_classification,
            )
        }
        continue
    
    # create HMMERprediction instance to represent the prediction for the current working protein
    cazyme_classification = 1
    parent_cazyme = CazymeProteinPrediction("HMMER", protein_accession, cazyme_classification)

    # get prediction for HMMER and separate the predicted CAZyme domains
    hmmer_predictions = line[1].split("+")
    
    # create empty dictionary to store CAZyme domains {family: HMMERCazymeDomain instance}
    cazyme_domains = {}
    
    # parse each of the predicted CAZyme domains
    for predicted_domain in hmmer_predictions:

        # separate the CAZy family and amino acid domain range
        predicted_domain = predicted_domain.split("(")
        
        # get the CAZy (sub)family of the predicted domain
        domain_name = predicted_domain[0]
        
        # check if an unusal CAZy family format  or CAZy subfamily is given
        if domain_name.find("_") != -1:  # found in unusal name formating or CAZy subfamily
        
            try:
                re.match(r"\D{2,3}\d+?_\D", domain_name).group()
                cazy_family = domain_name[:domain_name.find("_")]
                cazy_subfamily = np.nan

            except AttributeError:  # raised if not unusal format
                
                try:
                    # check it is a CAZy subfamily
                    re.match(r"\D{2,3}\d+?_\d+", domain_name).group()
                    cazy_family = domain_name[:domain_name.find("_")]
                    cazy_subfamily = np.nan
                
                except AttributeError:  # raised if doesn't match expected CAZy subfamily
                    print(
                        f"WARNING -- UNKNOWN DATA TYPE OF {domain_name} FOR {protein_accession}\n"
                        "Not adding as a domain."
                    )
                    continue
        
        else:
            # potentially have a CAZy family
            try:
                re.match(r"\D{2,3}\d+", domain_name).group()
                cazy_family = domain_name
                cazy_subfamily = np.nan
                
            except AttributeError:  # raised if doesn't match expected CAZy family
                print(
                    f"WARNING -- UNKNOWN DATA TYPE OF {domain_name} FOR {protein_accession}\n"
                    "Not adding as a domain."
                )
                continue

        # get the predicted amino acid domain range
        domain_range = predicted_domain[1][:-1] # exclude the final ")"
        # check domain_range is in the correct format
        try:
            re.match(r"\d+?-\d+", domain_range).group()
        except AttributeError:  # raised if not the correct format
            print(
                f"WARNING -- UNKNOWN DATA TYPE OF {domain_range} FOR {protein_accession}\n"
                f"Not adding domain range for the domain {cazy_family}-{cazy_subfamily}"
            )
            domain_range = np.nan
        
        # create identifier for the CAZyme domain to check if it has already been catalogued
        domain_identifier = f"{cazy_family}-{cazy_subfamily}"
        
        # check if the CAZyme domain has already been cataglouged
        try:
            existing_domain = cazyme_domains[domain_identifier]
            
            # CAZyme domain already catalogued, check if the retrieved domain range is the same as in the instance
            if domain_range not in existing_domain.domain_range:
                # append the new domain_range
                existing_domain.domain_range.append(domain_range)
                print("multiple domain ranges=", existing_domain, existing_domain.domain_range, protein_accession)
        
        except KeyError:  # raised if the CAZyme domain has not yet been catalogued
            cazyme_domains[domain_identifier] = CazymeDomain(
                prediction_tool="HMMER",
                protein_accession=protein_accession,
                cazy_family=cazy_family,
                cazy_subfamily=cazy_subfamily,
                domain_range=[domain_range],
            )
    
    # add CAZyme domains to the parent CAZyme
    parent_cazyme.cazyme_domains = list(cazyme_domains.values())
    
    # add the protein to the dictionary of all parsed proteins
    try:
        overview_dict[protein_accession]
        
        try:
            overview_dict[protein_accession]["HMMER"]
            print("WARNING -- PROTEIN ALREADY CATALOGUED WITH HMMER PREDICTION")
        
        except KeyError:  # raised it not HMMER prediction catagloued yet
            overview_dict[protein_accession]["HMMER"] = parent_cazyme
    
    except KeyError:  # raised if not protein yet catalogued
        overview_dict[protein_accession] = {"HMMER": parent_cazyme}
        
    print(parent_cazyme)

print("number of parsed proteins=", len(list(overview_dict.values())))

-CazymeProteinPrediction, protein=ATY58323.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58330.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58395.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58396.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58475.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58478.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58486.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58494.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58512.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58528.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58531.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58532.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58571.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58579.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58636.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY586

To make the code easier to read, we can pass out each of the different tasks (such as retrieving the subfamily, domain range, etc.) to separate functions.

Although it is not necessary to explicity write out the positional attributes of the CazymeDomain class, but it is neater to write out all attributes and explicitly assign their values, instead of only explicitly assigning the values of the optional attributes (i.e. `domain_range`)>

In [20]:
def parse_hmmer():
    """Parse data from HMMER presented in the overview.txt file"""
    # Check how text is separated in overview.txt
    with open ("overview.txt", "r") as ofh:
        overview_file = ofh.read().splitlines()

    overview_dict = {}

    for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):  # skip the first line becuase this is the 'column' titles
        line = line.split("\t")

        protein_accession = line[0]

        # check if the protein was classified as a non-CAZyme
        if line[1] == "-":
            cazyme_classification = 0  # non-CAZyme

            overview_dict[protein_accession] = {
                "HMMER": CazymeProteinPrediction(
                    "HMMER",
                    protein_accession,
                    cazyme_classification,
                )
            }
            continue

        # create HMMERprediction instance to represent the prediction for the current working protein
        cazyme_classification = 1
        parent_cazyme = CazymeProteinPrediction("HMMER", protein_accession, cazyme_classification)

        # get prediction for HMMER and separate the predicted CAZyme domains
        hmmer_predictions = line[1].split("+")

        # create empty dictionary to store CAZyme domains {family: HMMERCazymeDomain instance}
        cazyme_domains = get_cazyme_domains(hmmer_predictions, protein_accession)

        # add CAZyme domains to the parent CAZyme
        parent_cazyme.cazyme_domains = list(cazyme_domains.values())

        # add the protein to the dictionary of all parsed proteins
        try:
            overview_dict[protein_accession]

            try:
                overview_dict[protein_accession]["HMMER"]
                print("WARNING -- PROTEIN ALREADY CATALOGUED WITH HMMER PREDICTION")

            except KeyError:  # raised it not HMMER prediction catagloued yet
                overview_dict[protein_accession]["HMMER"] = parent_cazyme

        except KeyError:  # raised if not protein yet catalogued
            overview_dict[protein_accession] = {"HMMER": parent_cazyme}

        print(parent_cazyme)

    return overview_dict


def get_cazyme_domains(hmmer_predictions, protein_accession):
    """Retrieve predicted CAZyme domains, predited to be in a CAZyme by HMMER
    
    :param hmmer_predictions: list, data from HMMER from overview.txt file
    :param protein_accession: str
    
    Return dictionary, keyed by protein accession and valued by CazymeDomain instance.
    """
    cazyme_domains = {}

    # parse each of the predicted CAZyme domains
    for predicted_domain in hmmer_predictions:

        # separate the CAZy family and amino acid domain range
        predicted_domain = predicted_domain.split("(")

        # get the CAZy (sub)family of the predicted domain
        domain_name = predicted_domain[0]

        # check if an unusal CAZy family format  or CAZy subfamily is given
        if domain_name.find("_") != -1:  # found in unusal name formating or CAZy subfamily

            try:
                re.match(r"\D{2,3}\d+?_\D", domain_name).group()
                cazy_family = domain_name[:domain_name.find("_")]
                cazy_subfamily = np.nan

            except AttributeError:  # raised if not unusal format

                try:
                    # check it is a CAZy subfamily
                    re.match(r"\D{2,3}\d+?_\d+", domain_name).group()
                    cazy_family = domain_name[:domain_name.find("_")]
                    cazy_subfamily = np.nan

                except AttributeError:  # raised if doesn't match expected CAZy subfamily
                    print(
                        f"WARNING -- UNKNOWN DATA TYPE OF {domain_name} FOR {protein_accession}\n"
                        "Not adding as a domain."
                    )
                    continue

        else:
            # potentially have a CAZy family
            try:
                re.match(r"\D{2,3}\d+", domain_name).group()
                cazy_family = domain_name
                cazy_subfamily = np.nan

            except AttributeError:  # raised if doesn't match expected CAZy family
                print(
                    f"WARNING -- UNKNOWN DATA TYPE OF {domain_name} FOR {protein_accession}\n"
                    "Not adding as a domain."
                )
                continue

        # get the predicted amino acid domain range
        domain_range = predicted_domain[1][:-1] # exclude the final ")"
        # check domain_range is in the correct format
        try:
            re.match(r"\d+?-\d+", domain_range).group()
        except AttributeError:  # raised if not the correct format
            print(
                f"WARNING -- UNKNOWN DATA TYPE OF {domain_range} FOR {protein_accession}\n"
                f"Not adding domain range for the domain {cazy_family}-{cazy_subfamily}"
            )
            domain_range = np.nan

        # create identifier for the CAZyme domain to check if it has already been catalogued
        domain_identifier = f"{cazy_family}-{cazy_subfamily}"

        # check if the CAZyme domain has already been cataglouged
        try:
            existing_domain = cazyme_domains[domain_identifier]

            # CAZyme domain already catalogued, check if the retrieved domain range is the same as in the instance
            if domain_range not in existing_domain.domain_range:
                # append the new domain_range
                existing_domain.domain_range.append(domain_range)
                print("multiple domain ranges=", existing_domain, existing_domain.domain_range, protein_accession)

        except KeyError:  # raised if the CAZyme domain has not yet been catalogued
            cazyme_domains[domain_identifier] = CazymeDomain(
                prediction_tool="HMMER",
                protein_accession=protein_accession,
                cazy_family=cazy_family,
                cazy_subfamily=cazy_subfamily,
                domain_range=[domain_range],
            )

    return cazyme_domains

overview_dict = parse_hmmer()
print("number of parsed proteins=", len(list(overview_dict.values())))

-CazymeProteinPrediction, protein=ATY58323.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58330.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58395.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58396.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58475.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58478.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58486.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58494.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58512.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58528.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58531.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58532.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58571.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58579.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY58636.1, CAZyme domains=1-
-CazymeProteinPrediction, protein=ATY586

<a id="hotpep"></a>

## Hotpep

For Hotpep the data we want to retrieve is:
- The protein accession (which is the 'Gene ID')
- The CAZy (sub)families HMMER has predicted CAZy would catalogue the protein under
- Predicted EC numbers of the predicted CAZy domains (each domain is represented as a CAZy family prediction, thus predicting two CAZy families indicates predicting two CAZyme domains)

<a id="over_diamond"></a>

### Parsing Hotpep data from overview.txt

For Hotpep the data we want to retrieve is:

- The protein accession (which is the 'Gene ID')
- The CAZy (sub)families HMMER has predicted CAZy would catalogue the protein under
- Predicted EC numbers of the predicted CAZy domains (each domain is represented as a CAZy family prediction, thus predicting two CAZy families indicates predicting two CAZyme domains)

However, on the CAZy family and subfamily can be retrieved from the overview.txt file.
To start lets look at how the Hotpep data is presented in the overview.txt file.

In [21]:
# Check how text is separated in overview.txt
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()

for line in overview_file[1:]:  # skip the first line becuase this is the 'column' titles
    line = line.split("\t")
    hotpep = line[2]
    print(line[0], hotpep)


ATY58323.1 -
ATY58330.1 AA11(2)
ATY58395.1 -
ATY58396.1 AA6(1)
ATY58475.1 AA3(6)
ATY58478.1 GH16(14)
ATY58486.1 GH18(92)+CBM1(13)+CBM19(1)
ATY58494.1 GH5(55)
ATY58512.1 GH55(5)
ATY58528.1 -
ATY58531.1 GH64(5)+CBM6(29)
ATY58532.1 -
ATY58571.1 GH76(7)
ATY58579.1 GH38(5)
ATY58636.1 -
ATY58637.1 -
ATY58653.1 -
ATY58674.1 GH75(2)
ATY58687.1 -
ATY58705.1 GH29(53)
ATY58803.1 GT66(1)
ATY58848.1 GH18(42)+CBM12(19)+CBM16(5)+CBM5(17)
ATY58880.1 GH1(8)
ATY58895.1 AA3(18)
ATY58902.1 AA3(14)
ATY58910.1 -
ATY58915.1 -
ATY58949.1 GH16(41)
ATY58965.1 -
ATY59023.1 GH18(45)+CBM18(6)
ATY59031.1 GH13(23)+CBM20(17)
ATY59065.1 GH81(1)+CBM6(13)
ATY59093.1 GH18(51)+CBM18(12)+CBM5(14)
ATY59185.1 GT90(8)
ATY59188.1 GH18(102)+CBM18(5)
ATY59194.1 GH72(3)
ATY59202.1 -
ATY59250.1 GT15(1)
ATY59363.1 AA12(1)
ATY59524.1 -
ATY59596.1 -
ATY59638.1 GT21(11)
ATY59663.1 -
ATY59671.1 -
ATY59673.1 -
ATY59688.1 AA1(4)
ATY59691.1 GT1(76)
ATY59698.1 GH54(1)+CBM13(36)+CBM42(1)
ATY59714.1 GT4(169)
ATY59728.1 -
ATY59756.1 -
ATY5977

Immediately, after the CAZy family is the number of the k-mer group of the CAZy family in the Hotpep, and this number is not needed. The k-mer group number is written in parentheses. This allows an easy separation of the CAZy family/subfamily from the k-mer group number.

Hotpep can predict a protein has multiple CAZyme domains. Each CAZyme domain in a protein is identified by predicting a different CAZy (sub)family). For example, predicting the CAZy (sub)families GH3_1 and CBM2 in a protein indicates the protein has two CAZyme domains, one belonging to the CAZy family GH3, subfamily GH3_1 and the other domain belongs to the CAZy family CBM2. Each CAZyme domain will be stored in a `DIAMONDCazymeDomain` instance, which will then be stored in a parent `DIAMONDprediction` instance, which stores all the data for the parent protein.

Additionally, if Hotpep predicts a protein is not a CAZyme only "-" is given.

As with parsing HMMER, the each protein is stored as a HOTPEPprediction instance, stored in a dictionary keyed by the protein accession and valued by a dictionary, containing the key `Hotpep` and valued by a HOTPEPprediction instance.

In [22]:
# Check how text is separated in overview.txt
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()

# create an empty dictionary store all dbCAN, HMMER, Hotpep and DIAMOND prediction data
# {protein_accession: {prediciton_tool: CazymeProteinPrediction}}
overview_dict = {}

for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):  # skip the first line becuase this is the 'column' titles
    line = line.split("\t")
    
    # retrieve and separate the Hotpep CAZyme domains
    hotpep_predictions = line[2].split("+")

    protein_accession = line[0]
    
    # check if the protein was classified as a non-CAZyme
    if hotpep_predictions[0] == "-":
        cazyme_classification = 0
        
        try:
            overview_dict[protein_accession]
            
            try:
                overview_dict[protein_accession]["Hotpep"]
                
                print(f"WARNING -- HOTPEP PREDICTION ALREADY PROVIDED FOR {protein_accession}")
            
            except KeyError:  # raised if Hotpep prediction not yet catalogued for the protein
                overview_dict[protein_accession]["Hotpep"] = CazymeProteinPrediction(
                    "Hotpep",
                    protein_accesion,
                    cazyme_classification,
                )
        
        except KeyError:  # raised if protein is not yet catalogued
            overview_dict[protein_accession] = {
                "Hotpep": CazymeProteinPrediction(
                    "Hotpep",
                    protein_accession,
                    cazyme_classification,
                )
            }
        
        continue  # go to next protein
    
    # create CazymeProteinPrediction instance to represent the prediction for the current working protein
    cazyme_classification = 1
    parent_cazyme = CazymeProteinPrediction("Hotpep", protein_accession, cazyme_classification)
    
    cazyme_domains = []

    # parse each of the predicted CAZyme domains
    for predicted_domain in hotpep_predictions:

        # remove k-mer group number
        predicted_domain = predicted_domain.split("(")[0]

        # check if CAZy subfamily is predicted
        if predicted_domain.find("_") != -1:  # found CAZy subfamily
            cazy_family = predicted_domain[:predicted_domain.find("_")]
            cazy_subfamily = predicted_domain
        
        else:  # found CAZy family
            cazy_family = predicted_domain
            cazy_subfamily = np.nan
        
        # create DIAMONDCazymeDomain
        domain = CazymeDomain(
            prediction_tool="Hotpep",
            protein_accession="protein_accession",
            cazy_family="cazy_family",
            cazy_subfamily="cazy_subfamily",
        )

        cazyme_domains.append(domain)
    
    # add domains to the protein
    parent_cazyme.cazyme_domains = cazyme_domains
    
    # add the protein to the dictionary of all parsed proteins
    try:
        overview_dict[protein_accession]
        
        try:
            overview_dict[protein_accession]["HOTPEP"]

            print(f"WARNING -- HOTPEP PREDICTION ALREADY PROVIDED FOR {protein_accession}")
        
        except KeyError:  # raised it not HMMER prediction catagloued yet
            overview_dict[protein_accession]["HOTPEP"] = parent_cazyme
    
    except KeyError:  # raised if not protein yet catalogued
        overview_dict[protein_accession] = {"HOTPEP": parent_cazyme}
        
    print("resulting protein=", parent_cazyme)

print("number of parsed proteins=", len(list(overview_dict.values())))

resulting protein= -CazymeProteinPrediction, protein=ATY58330.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY58396.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY58475.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY58478.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY58486.1, CAZyme domains=3-
resulting protein= -CazymeProteinPrediction, protein=ATY58494.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY58512.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY58531.1, CAZyme domains=2-
resulting protein= -CazymeProteinPrediction, protein=ATY58571.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY58579.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY58674.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY58705.1, CAZyme domains=1-
resu

resulting protein= -CazymeProteinPrediction, protein=ATY64700.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY64752.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY64802.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY64843.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY64850.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY64919.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY64924.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY64953.1, CAZyme domains=2-
resulting protein= -CazymeProteinPrediction, protein=ATY64976.1, CAZyme domains=2-
resulting protein= -CazymeProteinPrediction, protein=ATY64996.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY65020.1, CAZyme domains=1-
resulting protein= -CazymeProteinPrediction, protein=ATY65055.1, CAZyme domains=1-
resu

By comparing the progress bar and the `number of parsed proteins` we can see that the method above parses all proteins catalogued in the overview.txt.

Additionally, by looking at the CazymeProteinPrediction instances we can see that the multilpe domains of the proteins are captured. For example, Hotpep predicted the protein ATY58848.1 contained `GH18(42)+CBM12(19)+CBM16(5)+CBM5(17)`, and this produced the output from above: `-CazymeProteinPrediction, protein=ATY58848.1, CAZyme domains=4-`

However, the predicted EC numbers are stored in the 'Hotpep.out' file. It is not guarenteed that the proteins will be listed in same order as they listed in the 'overview.txt' file. Therefore, instead of adding all the parsed proteins from the 'overview.txt' file to the `overview_dict` dictionary, but instead to another dictionary keyed by the protein accession and valued by the corresponding HOTPEPprediction instance.

Then the 'Hotpep.out' file will be parsed to retrieve the predicted EC numbers in each row.

<a id="hotpep_out"></a>

### Parsing the Hotpep.out file

The Hotpep.out file is parsed to retrieve the predicted EC numbers for each protein. Each row contains a unqiue protein. Once the EC numbers are retrieved for a protein the corresponding protein can be found in the `hotpep_proteins` dict, which is keyed by protein accession and valued by the corresponding `HOTPEPprediction` instance. Once the corresponding `HOTPEPprediction` instance is found, the corresponding EC numbers can be added.

Going to be a previous section in the notebook, the 'Gene ID' column the 'Hotpep.out' file contains the protein accession and the column to the furthest right contains the predicted EC numbers.

In [23]:
with open ("Hotpep.out", "r") as ofh:
    hotpep_file = ofh.read().splitlines()
    
for line in hotpep_file[:20]:
    line = line.split("\t")
    
    protein_accession = line[2]
    ec_numbers = line[-1]
    print(protein_accession, "**", ec_numbers)

Gene ID ** EC number
ATY65081.1 ** 3.1.1.6:22
ATY62702.1 ** NA
ATY63454.1 ** NA
ATY64953.1 ** NA
ATY65055.1 ** 3.1.1.74:26
ATY66875.1 ** 3.5.1.25:14
ATY64043.1 ** NA
ATY60133.1 ** NA
ATY62132.1 ** NA
ATY63707.1 ** NA
ATY65793.1 ** NA
ATY62951.1 ** NA
ATY61188.1 ** NA
ATY62721.1 ** NA
ATY65605.1 ** NA
ATY58880.1 ** 3.2.1.21:2152, 3.2.1.-:269, 3.2.1.38:49, 3.2.1.23:60, 3.2.1.125:16, 3.2.1.25:39, 3.2.1.182:12, 3.2.1.74:34, 3.2.1.118:33
ATY66369.1 ** NA
ATY66363.1 ** NA
ATY66542.1 ** NA


Looking at the data above, and referencing back to the early section of the notebook which examined the 'Hotpep.out' file, every listed EC number is immediately followed by a colon and the group number of the EC number in Hotpep, which is not a number we need.

Additionally, if multiple EC numbers are predicted they are separated by commas, making it very easy to separate them by using the `split` method.

However, we need to check if the 'Hotpep.out' file contains a unique protein per line. To do this, lets check how many lines there for the protein with the accession 'ATY58486.1' because in the 'overview.txt' file shows that Hotpep predicted the protein contains 3 CAZyme domains.

In [24]:
protein_of_interest = "ATY58486.1"
with open ("Hotpep.out", "r") as ofh:
    hotpep_file = ofh.read().splitlines()
    
for line in hotpep_file:
    line = line.split("\t")
    
    protein_accession = line[2]
    ec_numbers = line[-1]
    cazy_fam = line[0]
    if protein_accession == protein_of_interest:
        print(protein_accession, cazy_fam, "**", ec_numbers)

ATY58486.1 GH18 ** 3.2.1.14:15
ATY58486.1 CBM1 ** NA
ATY58486.1 CBM19 ** 3.2.1.14:125


We can see that each CAZyme domain predicted to be present in a protein is presented on a separate line, indicated by the same protein accession and different CAZy family. Therefore, we can use this (by identifying the unqiue CAZy family - CAZy subfamily pairing of a CAZyme domain) to store the predicted EC numbers within their respective `CazymeDomain` instance, for the parent CAZyme.

We need to retrieve the CAZyme domains from the corresponding `CazymeProteinPrediction` instance, and check which of the `CazymeDomain` instances match the current working CAZyme domain from the 'Hotpep.out' file.

Last we also need to check what happens to proteins Hotpep has predicted are not CAZymes, such as "ATY58323.1".

In [25]:
protein_of_interest = "ATY58323.1"
with open ("Hotpep.out", "r") as ofh:
    hotpep_file = ofh.read().splitlines()
    
for line in hotpep_file:
    line = line.split("\t")
    
    protein_accession = line[2]
    ec_numbers = line[-1]
    cazy_fam = line[0]
    if protein_accession == protein_of_interest:
        print(protein_accession, cazy_fam, "**", ec_numbers)

We can see that proteins Hotpep has predicted to be non-CAZymes are not included in the 'Hotpep.out' file.

Just before combining all the data and parsing of the Hotpep data, lets design a method to get the EC numbers from the 'Hotpep.out' file. The EC numbers want to be retrieved as a list of strings where each string contains a sinlge EC number becuase this will make it easier to select specific EC numbers if required later.

Furthermore, if no EC numbers are stored the string "NA" is written instead, which will be converted to a true null value. Also, we can adding in quality checking using regular expressions to ensure what is believed to be an EC number is an EC number.

In [26]:
with open ("Hotpep.out", "r") as ofh:
    hotpep_file = ofh.read().splitlines()
    
for line in hotpep_file:
    line = line.split("\t")
    
    protein_accession = line[2]
    
    # retrieve the EC numbers as a list
    ec_numbers = line[-1]
    ec_numbers = ec_numbers.split(",")
    # strip any remaining white space becuase the separating comma
    # is sometimes followed by a space
    index = 0
    for index in range(len(ec_numbers)):
        ec_numbers[index] = ec_numbers[index].strip()
        ec_numbers[index] = ec_numbers[index].split(":")[0]
        if ec_numbers[index] == "NA":
            ec_numbers[index] = np.nan
    
    print(protein_accession, cazy_fam, ec_numbers)


Gene ID CBM67 ['EC number']
ATY65081.1 CBM67 ['3.1.1.6']
ATY62702.1 CBM67 [nan]
ATY63454.1 CBM67 [nan]
ATY64953.1 CBM67 [nan]
ATY65055.1 CBM67 ['3.1.1.74']
ATY66875.1 CBM67 ['3.5.1.25']
ATY64043.1 CBM67 [nan]
ATY60133.1 CBM67 [nan]
ATY62132.1 CBM67 [nan]
ATY63707.1 CBM67 [nan]
ATY65793.1 CBM67 [nan]
ATY62951.1 CBM67 [nan]
ATY61188.1 CBM67 [nan]
ATY62721.1 CBM67 [nan]
ATY65605.1 CBM67 [nan]
ATY58880.1 CBM67 ['3.2.1.21', '3.2.1.-', '3.2.1.38', '3.2.1.23', '3.2.1.125', '3.2.1.25', '3.2.1.182', '3.2.1.74', '3.2.1.118']
ATY66369.1 CBM67 [nan]
ATY66363.1 CBM67 [nan]
ATY66542.1 CBM67 [nan]
ATY65191.1 CBM67 [nan]
ATY62175.1 CBM67 ['3.2.1.20', '2.4.1.-', '3.2.1.28', '3.2.1.10', '3.2.1.-', '3.2.1.133']
ATY59031.1 CBM67 ['3.2.1.1', '3.2.1.13.2.1']
ATY61965.1 CBM67 ['3.2.1.1']
ATY67289.1 CBM67 ['2.4.1.18']
ATY62092.1 CBM67 ['3.2.1.-']
ATY61276.1 CBM67 ['2.4.1.25', '3.2.1.33']
ATY60620.1 CBM67 [nan]
ATY62564.1 CBM67 ['3.2.1.3']
ATY61850.1 CBM67 [nan]
ATY60381.1 CBM67 [nan]
ATY62739.1 CBM67 [nan]
AT

Therefore, first the 'overview.txt' file must be parsed and then the 'Hotpep.out' file. The following functions are invoked before adding the proteins to the overview_dict.

In [27]:
def parse_overview():
    # Check how text is separated in overview.txt
    with open ("overview.txt", "r") as ofh:
        overview_file = ofh.read().splitlines()

    # create an empty dictionary store all dbCAN, HMMER, Hotpep and DIAMOND prediction data
    # {protein_accession: {prediciton_tool: CazymeProteinPrediction}}
    overview_dict = {}
    
    hotpep_overview_dict = {}

    for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):  # skip the first line becuase this is the 'column' titles
        line = line.split("\t")

        # retrieve and separate the Hotpep CAZyme domains
        hotpep_predictions = line[2].split("+")

        protein_accession = line[0]

        # check if the protein was classified as a non-CAZyme
        if hotpep_predictions[0] == "-":
            cazyme_classification = 0

            try:
                overview_dict[protein_accession]

                try:
                    overview_dict[protein_accession]["Hotpep"]

                    print(f"WARNING -- HOTPEP PREDICTION ALREADY PROVIDED FOR {protein_accession}")

                except KeyError:  # raised if Hotpep prediction not yet catalogued for the protein
                    overview_dict[protein_accession]["Hotpep"] = CazymeProteinPrediction(
                        "Hotpep",
                        protein_accesion,
                        cazyme_classification,
                    )

            except KeyError:  # raised if protein is not yet catalogued
                overview_dict[protein_accession] = {
                    "Hotpep": CazymeProteinPrediction(
                        "Hotpep",
                        protein_accession,
                        cazyme_classification,
                    )
                }

            continue  # go to next protein

        # create CazymeProteinPrediction instance to represent the prediction for the current working protein
        cazyme_classification = 1
        parent_cazyme = CazymeProteinPrediction("Hotpep", protein_accession, cazyme_classification)

        cazyme_domains = []

        # parse each of the predicted CAZyme domains
        for predicted_domain in hotpep_predictions:

            # remove k-mer group number
            predicted_domain = predicted_domain.split("(")[0]

            # check if CAZy subfamily is predicted
            if predicted_domain.find("_") != -1:  # found CAZy subfamily
                cazy_family = predicted_domain[:predicted_domain.find("_")]
                cazy_subfamily = predicted_domain

            else:  # found CAZy family
                cazy_family = predicted_domain
                cazy_subfamily = np.nan

            # create DIAMONDCazymeDomain
            domain = CazymeDomain(
                prediction_tool="Hotpep",
                protein_accession="protein_accession",
                cazy_family="cazy_family",
                cazy_subfamily="cazy_subfamily",
            )

            cazyme_domains.append(domain)

        # add domains to the protein
        parent_cazyme.cazyme_domains = cazyme_domains

        # add the protein to the dictionary of all parsed proteins
        try:
            hotpep_overview_dict[protein_accession]

            print(f"WARNING -- HOTPEP PREDICTION ALREADY PROVIDED FOR {protein_accession}")

        except KeyError:  # raised if not protein yet catalogued
            hotpep_overview_dict[protein_accession] = parent_cazyme

    return overview_dict, hotpep_overview_dict

overview_dict, hotpep_overview_dict = parse_overview()            
print("non-cazymes=", len(list(overview_dict.values())))
print("predicted cazymes=", len(list(hotpep_overview_dict.values())))
print("total=", (len(list(overview_dict.values())) + len(list(hotpep_overview_dict.values()))))


non-cazymes= 193
predicted cazymes= 252
total= 445


In [28]:
def parse_hotpep_file(hotpep_overview_dict):
    with open ("Hotpep.out", "r") as ofh:
        hotpep_file = ofh.read().splitlines()

    for line in hotpep_file[1:]:  # skip the first line, which contains titles
        line = line.split("\t")

        protein_accession = line[2].strip()

        # retrieve the EC numbers as a list
        ec_numbers = line[-1]
        ec_numbers = ec_numbers.split(",")
        # strip any remaining white space becuase the separating comma
        # is sometimes followed by a space
        index = 0
        for index in range(len(ec_numbers)):
            ec_numbers[index] = ec_numbers[index].strip()
            ec_numbers[index] = ec_numbers[index].split(":")[0]
            if ec_numbers[index] == "NA":
                ec_numbers[index] = np.nan
        
        # get CAZy family and subfamily of the current working domain
        cazy_family = line[0]
        if cazy_family.find("_") != -1:
            cazy_subfamily = cazy_family
            cazy_family = cazy_family[:cazy_family.find("_")]
        else:
            cazy_subfamily = np.nan

        domain_identifer = [cazy_family, cazy_subfamily]  # used to find the correct domain

        # get the list of CAZyme domains of the parent protein
        parent_protein = hotpep_overview_dict[protein_accession]
        parent_domains = parent_protein.cazyme_domains
        
        for domain in parent_domains:
            if domain_identifer == [domain.cazy_family, domain.cazy_subfamily]:
                domain.ec_numbers = ec_numbers

parse_hotpep_file(hotpep_overview_dict)
"done"

'done'

Now we can combine our method of parsing the 'overview.txt' file, and the 'Hotpep.out' to populate the CazymeDomain instances with EC numbers. Additionally, to make the code more human-readable, we can break up the tasks (retrieving EC numbers, parsing different files, etc.) into separate functions.

In [29]:
def parse_overview():
    # Check how text is separated in overview.txt
    with open ("overview.txt", "r") as ofh:
        overview_file = ofh.read().splitlines()

    # create an empty dictionary store all dbCAN, HMMER, Hotpep and DIAMOND prediction data
    # {protein_accession: {prediciton_tool: CazymeProteinPrediction}}
    overview_dict = {}
    
    hotpep_overview_dict = {}

    for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):  # skip the first line becuase this is the 'column' titles
        line = line.split("\t")

        # retrieve and separate the Hotpep CAZyme domains
        hotpep_predictions = line[2].split("+")

        protein_accession = line[0]

        # check if the protein was classified as a non-CAZyme
        if hotpep_predictions[0] == "-":
            cazyme_classification = 0

            try:
                overview_dict[protein_accession]

                try:
                    overview_dict[protein_accession]["Hotpep"]

                    print(f"WARNING -- HOTPEP PREDICTION ALREADY PROVIDED FOR {protein_accession}")

                except KeyError:  # raised if Hotpep prediction not yet catalogued for the protein
                    overview_dict[protein_accession]["Hotpep"] = CazymeProteinPrediction(
                        "Hotpep",
                        protein_accesion,
                        cazyme_classification,
                    )

            except KeyError:  # raised if protein is not yet catalogued
                overview_dict[protein_accession] = {
                    "Hotpep": CazymeProteinPrediction(
                        "Hotpep",
                        protein_accession,
                        cazyme_classification,
                    )
                }

            continue  # go to next protein

        # create CazymeProteinPrediction instance to represent the prediction for the current working protein
        cazyme_classification = 1
        parent_cazyme = CazymeProteinPrediction("Hotpep", protein_accession, cazyme_classification)
        
        cazyme_domains = get_cazyme_domains(hotpep_predictions, protein_accession)  # list of CazymeDomain instances

        # add domains to the protein
        parent_cazyme.cazyme_domains = cazyme_domains

        # add the protein to the dictionary of all parsed proteins
        try:
            hotpep_overview_dict[protein_accession]

            print(f"WARNING -- HOTPEP PREDICTION ALREADY PROVIDED FOR {protein_accession}")

        except KeyError:  # raised if not protein yet catalogued
            hotpep_overview_dict[protein_accession] = parent_cazyme
        
    # populate predicted CAZyme domains with their EC number predictions
    hotpep_overview_dict = get_ec_numbers(hotpep_overview_dict)
    
    print("non-cazymes=", len(list(overview_dict.values())))
    print("predicted cazymes=", len(list(hotpep_overview_dict.values())))

    # add CAZymes to the overview_dict
    for protein_accession in hotpep_overview_dict:
        try:
            overview_dict[protein_accession]

            try:
                overview_dict[protein_accession]["Hotpep"]

                print(f"WARNING -- HOTPEP PREDICTION ALREADY PROVIDED FOR {protein_accession}")

            except KeyError:  # raised if Hotpep prediction not yet catalogued for the protein
                overview_dict[protein_accession]["Hotpep"] = hotpep_overview_dict[protein_accession]

        except KeyError:  # raised if protein is not yet catalogued
            overview_dict[protein_accession] = {"Hotpep": hotpep_overview_dict[protein_accession]}
    
    return list(overview_dict.values())

def get_cazyme_domains(hotpep_predictions, protein_accession):
    """Retrieve CAZyme domains predicted by Hotpep"""
    cazyme_domains = []
    # parse each of the predicted CAZyme domains
    for predicted_domain in hotpep_predictions:

        # remove k-mer group number
        predicted_domain = predicted_domain.split("(")[0]

        # check if CAZy subfamily is predicted
        if predicted_domain.find("_") != -1:  # found CAZy subfamily
            cazy_family = predicted_domain[:predicted_domain.find("_")]
            cazy_subfamily = predicted_domain

        else:  # found CAZy family
            cazy_family = predicted_domain
            cazy_subfamily = np.nan

        # create DIAMONDCazymeDomain
        domain = CazymeDomain(
            prediction_tool="Hotpep",
            protein_accession="protein_accession",
            cazy_family="cazy_family",
            cazy_subfamily="cazy_subfamily",
        )

        cazyme_domains.append(domain)
    
    return cazyme_domains


def get_ec_numbers(hotpep_overview_dict):
    """Retrieve EC numbers for each CAZyme domain from the Hotpep.out file"""
    with open ("Hotpep.out", "r") as ofh:
        hotpep_file = ofh.read().splitlines()

    for line in hotpep_file[1:]:  # skip the first line, which contains titles
        line = line.split("\t")

        protein_accession = line[2].strip()

        # retrieve the EC numbers as a list
        ec_numbers = line[-1]
        ec_numbers = ec_numbers.split(",")
        # strip any remaining white space becuase the separating comma
        # is sometimes followed by a space
        index = 0
        for index in range(len(ec_numbers)):
            ec_numbers[index] = ec_numbers[index].strip()
            ec_numbers[index] = ec_numbers[index].split(":")[0]
            if ec_numbers[index] == "NA":
                ec_numbers[index] = np.nan
        
        # get CAZy family and subfamily of the current working domain
        cazy_family = line[0]
        if cazy_family.find("_") != -1:
            cazy_subfamily = cazy_family
            cazy_family = cazy_family[:cazy_family.find("_")]
        else:
            cazy_subfamily = np.nan

        domain_identifer = [cazy_family, cazy_subfamily]  # used to find the correct domain

        # get the list of CAZyme domains of the parent protein
        parent_protein = hotpep_overview_dict[protein_accession]
        parent_domains = parent_protein.cazyme_domains
        
        for domain in parent_domains:
            if domain_identifer == [domain.cazy_family, domain.cazy_subfamily]:
                domain.ec_numbers = ec_numbers

    return hotpep_overview_dict

all_proteins = parse_overview()
print("done.", len(all_proteins))

for p in all_proteins:
    print(p["Hotpep"])


non-cazymes= 193
predicted cazymes= 252
done. 445
-CazymeProteinPrediction, protein=ATY58323.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY58395.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY58528.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY58532.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY58636.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY58637.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY58653.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY58687.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY58910.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY58915.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY58965.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY59202.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY59524.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY59596.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY59663.1, non-CAZyme-
-CazymeProteinPrediction, protein=ATY59671.1, non-CAZyme-
-CazymeProteinPredict

We can see from printing out the analysis that all proteins were parsed from the overview.txt and Hotpep.out files. Additionally, from printing out the proteins in the returned list of items, we can see we still maintain storing the CAZyme domains within their respective parent CAZyme (`CazymeProteinPrediciton` instance)

<a id="diamond"></a>

## DIAMOND

The data from **DIAMOND** includes:
- The protein accession
- CAZy family of each domain
- CAZy subfamily of each domain (if not in a subfamily, the subfamily is listed as a null value)

<a id="over_diamond"></a>

### Parsing DIAMOND data from overview.txt

The last tool to whose output is listed in the overview.txt file is DIAMOND. The data we want to retrieve from DIAMOND is the classification of proteins as CAZymes and non-CAZymes, and the predict CAZy families and subfamilies of each of the CAZyme domains in each protein.

Each CAZyme domain in a protein is identified by predicting a different CAZy (sub)family). For example, predicting the CAZy (sub)families GH3_1 and CBM2 in a protein indicates the protein has two CAZyme domains, one belonging to the CAZy family GH3, subfamily GH3_1 and the other domain belongs to the CAZy family CBM2.

First lets look at the structure of the data in the DIAMOND column of the overview.txt file.

In [30]:
# Check how text is separated in overview.txt
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()

for line in overview_file[1:]:  # skip the first line becuase this is the 'column' titles
    line = line.split("\t")
    diamond = line[3]
    print(line[0], diamond)


ATY58323.1 CE5
ATY58330.1 AA11
ATY58395.1 -
ATY58396.1 -
ATY58475.1 AA3
ATY58478.1 GH16
ATY58486.1 CBM1+GH18
ATY58494.1 GH5_24
ATY58512.1 GH55
ATY58528.1 GH16
ATY58531.1 GH64
ATY58532.1 CE5
ATY58571.1 GH76
ATY58579.1 GH38
ATY58636.1 GT2
ATY58637.1 GH12
ATY58653.1 -
ATY58674.1 GH75
ATY58687.1 -
ATY58705.1 GH29
ATY58803.1 GT66
ATY58848.1 GH18
ATY58880.1 GH1
ATY58895.1 AA3_1
ATY58902.1 AA3_3
ATY58910.1 -
ATY58915.1 -
ATY58949.1 GH16
ATY58965.1 CE5
ATY59023.1 CBM18+GH18
ATY59031.1 GH13_5
ATY59065.1 GH81
ATY59093.1 GH18
ATY59185.1 GT90
ATY59188.1 CBM18+CBM50+CBM66+GH18
ATY59194.1 GH72
ATY59202.1 GT90
ATY59250.1 GT15
ATY59363.1 AA12
ATY59524.1 -
ATY59596.1 -
ATY59638.1 GT21
ATY59663.1 -
ATY59671.1 CE4
ATY59673.1 -
ATY59688.1 AA1_2
ATY59691.1 GT1
ATY59698.1 CBM42+GH54
ATY59714.1 GT4
ATY59728.1 -
ATY59756.1 -
ATY59775.1 -
ATY59781.1 GT1
ATY59860.1 AA3_1
ATY59890.1 CBM66+GH43_34
ATY59904.1 AA3_2
ATY59918.1 -
ATY59933.1 GT2
ATY59950.1 GH128
ATY59985.1 GH47
ATY60044.1 GT2
ATY60072.1 GH16
ATY60094

Like HMMER and Hotpep, if DIAMOND predicts a protein is not a CAZyme, only a dash is provided.

If multiple CAZyme domains are predicted, each domain is separated by '+'.

However, if a CAZy subfamily is predicted only the CAZy subfamily is predicted, and not the CAZy family. The evaluation by `pyrewton` of the performance of CAZyme prediction needs the CAZy family and not the subfamily. Therefore, a check is included to see if a CAZy subfamily is provided. If it is the parent CAZy family and subfamily wil be retrieved.

Each CAZyme domain is represented by a unique DIAMONDCazymeDomain instance, including the CAZy family and subfamily (a null value if not given), which will then be stored as members in the parent DIAMONDprediction instance that represents a single protein.

In [31]:
# Check how text is separated in overview.txt
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()

# create an empty dictionary store all dbCAN, HMMER, Hotpep and DIAMOND prediction data
# {protein_accession: {prediciton_tool: CazymeProteinPrediction}}
overview_dict = {}

for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):  # skip the first line becuase this is the 'column' titles
    line = line.split("\t")
    
    # retrieve and separate the DIAMOND CAZyme domains
    diamond_predictions = line[3].split("+")

    protein_accession = line[0]
    
    # check if the protein was classified as a non-CAZyme
    if diamond_predictions[0] == "-":
        cazyme_classification = 0
        
        try:
            overview_dict[protein_accession]
            
            try:
                overview_dict[protein_accession]["DIAMOND"]
                
                print(f"WARNING -- DIAMOND PREDICTION ALREADY PROVIDED FOR {protein_accession}")
            
            except KeyError:  # raised if DIAMOND prediction not yet catalogued for the protein
                overview_dict[protein_accession]["DIAMOND"] = CazymeProteinPrediction(
                    "DIAMOND",
                    protein_accession,
                    cazyme_classification,
                )
        
        except KeyError:  # raised if protein is not yet catalogued
            overview_dict[protein_accession] = {
                "DIAMOND": CazymeProteinPrediction(
                    "DIAMOND",
                    protein_accession,
                    cazyme_classification,
                )
            }
        
        continue  # go to next protein
    
    # create DIAMONDprediction instance to represent the prediction for the current working protein
    cazyme_classification = 1
    parent_cazyme = CazymeProteinPrediction("DIAMOND", protein_accession, cazyme_classification)
    
    cazyme_domains = []

    # parse each of the predicted CAZyme domains
    for predicted_domain in diamond_predictions:
        print("predicted domain=", predicted_domain)
        
        # check if CAZy subfamily is predicted
        if predicted_domain.find("_") != -1:  # found CAZy subfamily
            cazy_family = predicted_domain[:predicted_domain.find("_")]
            cazy_subfamily = predicted_domain
        
        else:  # found CAZy family
            cazy_family = predicted_domain
            cazy_subfamily = np.nan
        
        domain = CazymeDomain(protein_accession, cazy_family, cazy_subfamily)

        cazyme_domains.append(domain)
    
    # add domains to the protein
    parent_cazyme.cazyme_domains = cazyme_domains
    
    # add the protein to the dictionary of all parsed proteins
    try:
        overview_dict[protein_accession]
        
        try:
            overview_dict[protein_accession]["DIAMOND"]
            
            print(f"WARNING -- DIAMOND PREDICTION ALREADY PROVIDED FOR {protein_accession}")
        
        except KeyError:  # raised it not HMMER prediction catagloued yet
            overview_dict[protein_accession]["DIAMOND"] = parent_cazyme
    
    except KeyError:  # raised if not protein yet catalogued
        overview_dict[protein_accession] = {"DIAMOND": parent_cazyme}
        
    print("resulting protein=", parent_cazyme)

print("number of parsed proteins=", len(list(overview_dict.values())))

predicted domain= CE5
resulting protein= -CazymeProteinPrediction, protein=ATY58323.1, CAZyme domains=1-
predicted domain= AA11
resulting protein= -CazymeProteinPrediction, protein=ATY58330.1, CAZyme domains=1-
predicted domain= AA3
resulting protein= -CazymeProteinPrediction, protein=ATY58475.1, CAZyme domains=1-
predicted domain= GH16
resulting protein= -CazymeProteinPrediction, protein=ATY58478.1, CAZyme domains=1-
predicted domain= CBM1
predicted domain= GH18
resulting protein= -CazymeProteinPrediction, protein=ATY58486.1, CAZyme domains=2-
predicted domain= GH5_24
resulting protein= -CazymeProteinPrediction, protein=ATY58494.1, CAZyme domains=1-
predicted domain= GH55
resulting protein= -CazymeProteinPrediction, protein=ATY58512.1, CAZyme domains=1-
predicted domain= GH16
resulting protein= -CazymeProteinPrediction, protein=ATY58528.1, CAZyme domains=1-
predicted domain= GH64
resulting protein= -CazymeProteinPrediction, protein=ATY58531.1, CAZyme domains=1-
predicted domain= CE5
r

By comparing the progress bar and the number of parsed proteins we can see that the method above parses all proteins catalogued in the overview.txt.

Additionally, by looking at the `CazymeProteinPrediction` instances we can see that the multilpe domains of the proteins are captured, and stored within their respecetive parent cazyme represented by a `CazymeProteinPrediction` instance.

To improve the robustness of the method, 'quality' checks can be included to make sure a CAZy (sub)family is being retrieved. This is retrieved by using regular expressions to check the format of the retrieved data.

In [32]:
# Check how text is separated in overview.txt
with open ("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()

# create an empty dictionary store all dbCAN, HMMER, Hotpep and DIAMOND prediction data
# {protein_accession: {prediciton_tool: CazymeProteinPrediction}}
overview_dict = {}

for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):  # skip the first line becuase this is the 'column' titles
    line = line.split("\t")
    
    # retrieve and separate the DIAMOND CAZyme domains
    diamond_predictions = line[3].split("+")

    protein_accession = line[0]
    
    # check if the protein was classified as a non-CAZyme
    if diamond_predictions[0] == "-":
        cazyme_classification = 0
        
        try:
            overview_dict[protein_accession]
            
            try:
                overview_dict[protein_accession]["DIAMOND"]
                
                print(f"WARNING -- DIAMOND PREDICTION ALREADY PROVIDED FOR {protein_accession}")
            
            except KeyError:  # raised if DIAMOND prediction not yet catalogued for the protein
                overview_dict[protein_accession]["DIAMOND"] = CazymeProteinPrediction(
                    "DIAMOND",
                    protein_accession,
                    cazyme_classification,
                )
        
        except KeyError:  # raised if protein is not yet catalogued
            overview_dict[protein_accession] = {
                "DIAMOND": CazymeProteinPrediction(
                    "DIAMOND",
                    protein_accession,
                    cazyme_classification,
                )
            }
        
        continue  # go to next protein
    
    # create DIAMONDprediction instance to represent the prediction for the current working protein
    cazyme_classification = 1
    parent_cazyme = CazymeProteinPrediction("DIAMOND", protein_accession, cazyme_classification)
    
    cazyme_domains = []

    # parse each of the predicted CAZyme domains
    for predicted_domain in diamond_predictions:
        print("predicted domain=", predicted_domain)
        
        # check if CAZy subfamily is predicted
        try:
            re.match(r"\D{2,3}\d+?_\d+", predicted_domain).group()
            
            cazy_family = predicted_domain[:predicted_domain.find("_")]
            cazy_subfamily = predicted_domain
        
        except AttributeError:  # raised if predicted_domain is not a CAZy subfamily
            try:
                re.match(r"\D{2,3}\d+", predicted_domain).group()
                
                cazy_family = predicted_domain
                cazy_subfamily = np.nan
            
            except AttributeError:  # raised it no a CAZy family
                print(
                    f"WARNING -- UNEXPECTED DATA FORMAT {predicted_domain}\n"
                    f"Expected cazy family or subfamily but this is neither, not adding as a CAZyme domain to {protein_accession}"
                )
                continue
        
        domain = CazymeDomain(protein_accession, cazy_family, cazy_subfamily)

        cazyme_domains.append(domain)
    
    # add domains to the protein
    parent_cazyme.cazyme_domains = cazyme_domains
    
    # add the protein to the dictionary of all parsed proteins
    try:
        overview_dict[protein_accession]
        
        try:
            overview_dict[protein_accession]["DIAMOND"]
            
            print(f"WARNING -- DIAMOND PREDICTION ALREADY PROVIDED FOR {protein_accession}")
        
        except KeyError:  # raised it not HMMER prediction catagloued yet
            overview_dict[protein_accession]["DIAMOND"] = parent_cazyme
    
    except KeyError:  # raised if not protein yet catalogued
        overview_dict[protein_accession] = {"DIAMOND": parent_cazyme}
        
    print("resulting protein=", parent_cazyme)

print("number of parsed proteins=", len(list(overview_dict.values())))

predicted domain= CE5
resulting protein= -CazymeProteinPrediction, protein=ATY58323.1, CAZyme domains=1-
predicted domain= AA11
resulting protein= -CazymeProteinPrediction, protein=ATY58330.1, CAZyme domains=1-
predicted domain= AA3
resulting protein= -CazymeProteinPrediction, protein=ATY58475.1, CAZyme domains=1-
predicted domain= GH16
resulting protein= -CazymeProteinPrediction, protein=ATY58478.1, CAZyme domains=1-
predicted domain= CBM1
predicted domain= GH18
resulting protein= -CazymeProteinPrediction, protein=ATY58486.1, CAZyme domains=2-
predicted domain= GH5_24
resulting protein= -CazymeProteinPrediction, protein=ATY58494.1, CAZyme domains=1-
predicted domain= GH55
resulting protein= -CazymeProteinPrediction, protein=ATY58512.1, CAZyme domains=1-
predicted domain= GH16
resulting protein= -CazymeProteinPrediction, protein=ATY58528.1, CAZyme domains=1-
predicted domain= GH64
resulting protein= -CazymeProteinPrediction, protein=ATY58531.1, CAZyme domains=1-
predicted domain= CE5
r

<a id="dbcan"></a>

## dbCAN (consensus result)

The prediction result for consensus result of HMMER, DIAMOND, and Hotpep, where at least two of the tools agree upon the prediction, as  the practise of the dbCAN developers.

The data from dbCAN includes:
- The protein accession
- CAZy family of each domain
- CAZy subfamily of each domain (if not in a subfamily, the subfamily is listed as a null value)

Rembering back to the investigation of the 'overview.txt' file earlier, each row contains:
- the data for a unique protein
- the prediction data from HMMER, Hotpep and DIAMOND
- the number of tools that predicted the protein was a CAZymes

Therefore, we can need to produce the dbCAN consensus result of the data in each row of the 'overview.txt' file to get the consensus dbCAN result for each protein.

A consesnsus result is a result that the majoirty of tools agree upon, therefore, we can quickly filter for proteins with the consensus result that the protein is not a CAZyme by checking if the value of '#ofTools' is 1. If the value is 2 or 3, then we can check for a consensus results.

Therefore, we we need to do is parse each line of the 'overview.txt' file one, and retrieve the HMMER, Hotpep, DIAMOND and dbCAN prediction, instead of how it has been layed out so far in this notebook (where the 'overview.txt' file is parsed each for HMMER, Hotpep and DIAMOND). 

The first task is to combine the wrok conducted so far in this notebook to createa series of function, which retrieve for each line in the 'overview.txt' file the HMMER, Hotpep and DIAMOND prediciton.

<a id="retrieve_hmmer_hotpep_diamond"></a>

### Retrieving the HMMER, Hotpep and DIAMOND data for each line in the overview.txt file

Below are a set of functions to parse each line of the overview.txt file, instead of having to parse the file for each CAZyme prediction tool (HMMER, Hotpep and DIAMOND).

In [33]:
def parse_dbcan_output(overview_path, hotpep_file):
    """Parse the data in the 'overview.txt' files from dbCAN.
    
    The 'overview.txt' file contains the CAZyme predicion data from
    HMMER, Hotpep and DIAMOND. A dbCAN consensus result is retrieved
    by finding predictions that at least to of the prediction tools
    agree upon.
    
    Each protein from the input FASTA file containing all query protein
    sequences parsed by dbCAN is represented by a CazymeProteinprediction
    instance. Each predicted CAZyme domains in each predicted CAZyme is
    represented by a CazymeDomain instance, which are sotred within their
    parent CAZyme's CazymeProteinPrediction instance. A CazymeProteinPrediction
    instance is created for each protein in the inpu FASTA file parsed
    by dbCAN and for each CAZyme prediction tool: dbCAN, HMMER, Hotpep and
    DIAMOND.
    
    :param overview_path: Path, path to dbCAN output overview.txt file
    :param hotpep_file: Path, path to Hotpep.out file
    
    Return 4 lists, each containing the CazymeProteinPrediction instances for
    HMMER, Hotpep, DIAMOND and dbCAN.
    """
    # open the overview.txt file
    with open(overview_path, "r") as ofh:
        overview_file = ofh.read().splitlines()
    
    # create an empty dictionary to store the CazymeProteinPrediction instances
    # {protein_accesion: CazymeProteinPrediction_instance}
    overview_dict = {}

    # skip the the first line in the file, becuase it contains the 'column' headings
    for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):
        # separate the line content
        line = line.split("\t")
        
        protein_accession = line[0]
        
        # create a CazymeProteinPrediction instance each for HMMER, Hotpep and DIAMOND
        hmmer_prediction = get_hmmer_prediction(line[1], protein_accession)
        hotpep_prediction = get_hotpep_prediction(line[2], protein_accession)
        diamond_prediction = get_diamond_prediction(line[3], protein_accession)
        
        # get the dbCAN consensus result
        # dbcan_prediction = get_dbcan_consensus(hmmer_prediction, hotpep_prediction, diamond_prediction)
        
        # add the CazymeProteinPrediction instances to the overview_dict
        try:
            overview_dict[protein_accession]
            prediction_outputs = [["HMMER", hmmer_prediction], ["Hotpep", hotpep_prediction], ["DIAMOND", diamond_prediction], ["dbCAN", dbcan_prediction]]
            
            for prediction_tool in prediction_outputs:
                try:
                    overview_dict[protein_accession][prediction_tool[0]]
                    print(
                        f"WARNING -- Protein {protein_accession} already catalogued with {prediction_tool[0]}"
                    )
                except KeyError:  # raised if CazymeProteinPrediciton instance not stored for prediction_tool
                    overview_dict[protein_accession][prediction_tool[0]] = prediction_tool[1]
        
        except KeyError:  # raised if protein has not been added yet
            overview_dict[protein_accession] = {
                "HMMER": hmmer_prediction,
                "Hotpep": hotpep_prediction,
                "DIAMOND": diamond_prediction,
                "dbCAN": "dbcan_prediction",
            }
    
    # get EC numbers predicted by Hotpep
    overview_dict = get_hotpep_ec_numbers(overview_dict, hotpep_file)
    return overview_dict
        

def get_hmmer_prediction(hmmer_data, protein_accession):
    """Retrieve HMMER prediciton data from the dbCAN overview.txt file.
    
    :param hmmer_data: str, output data from HMMER written in the overview.txt file.
    :param protein_accession: str
    
    Return a CazymeProteinPrediction instance.
    """
    # check if the protein was classified as a non-CAZyme
    if hmmer_data == "-":
        cazyme_classification = 0  # non-CAZyme
        protein = CazymeProteinPrediction("HMMER", protein_accession, cazyme_classification)
        return protein
    
    # create a CazymeProteinPrediction instance to represent the CAZyme
    cazyme_classification = 1
    parent_cazyme = CazymeProteinPrediction("HMMER", protein_accession, cazyme_classification)

    # HMMER can predicted multiple CAZyme domaisn for a CAZyme
    # HMMER separates each of the predicted CAZyme domains by additiona symbols
    parent_cazyme.cazyme_domains = get_hmmer_cazyme_domains(hmmer_data.split("+"), protein_accession)
    
    return parent_cazyme

    
def get_hmmer_cazyme_domains(predicted_domains, protein_accession):
    """Retrieve the CAZyme domains predicted by HMMER.
    
    :param predicted_domains: list of strings, each string containing a CAZyme domain prediction
    :param protein_accession: str
    
    Return list of CazymeDomain instances. Each instance represents a unique CAZyme domain.
    """
    cazyme_domains = {}  # {"cazyfam-cazysubfam": CazymeDomain_instance}

    for domain in predicted_domains:
        # separate the predicted CAZy family from the domain range
        domain = domain.split("(")
        
        # get the predicted CAZy family(subfamily)
        domain_name = domain[0]  # CAZy (sub)family
        cazy_family, cazy_subfamily = get_hmmer_cazy_family(domain_name)
        
        if cazy_family is None:  # unexpected data format found by get_hmmer_cazy_family()
            continue
        
        # get the predicted amino acid domain range
        domain_range = domain[1][:-1]  # drop the terminal ")"
        # check it matches the expected format and thus is the predicted domain range
        try:
            re.match(r"\d+?-\d+", domain_range).group()
        except AttributeError:  # raised if not correct format
            print(
                f"WARNING -- UNKNOWN DATA TYPE OF {domain_range} FOR {protein_accession}\n"
                f"Not adding domain range for the domain {cazy_family}-{cazy_subfamily}"
            )
            domain_range = np.nan

        # create identifier for the CAZyme domain to check if it has already been catalogued
        domain_identifier = f"{cazy_family}-{cazy_subfamily}"
    
        # check if the CAZyme domain has already been cataglouged
        try:
            existing_domain = cazyme_domains[domain_identifier]

            # CAZyme domain already catalogued, check if the retrieved domain range is the same
            if domain_range not in existing_domain.domain_range:
                # append the new domain_range
                existing_domain.domain_range.append(domain_range)
                print("multiple domain ranges=", existing_domain, existing_domain.domain_range, protein_accession)

        except KeyError:  # raised if the CAZyme domain has not yet been catalogued
            cazyme_domains[domain_identifier] = CazymeDomain(
                prediction_tool="HMMER",
                protein_accession=protein_accession,
                cazy_family=cazy_family,
                cazy_subfamily=cazy_subfamily,
                domain_range=[domain_range],
            )

    return list(cazyme_domains.values())


def get_hmmer_cazy_family(domain_name):
    """Retrieve the name of the CAZyme domain, this is the predicted CAZy family and subfamily.
    
    If a subfamily is not predicted, the CAZy subfamily is set as a null value.
    If the data formate does not match the expected format, the CAZy family will
    be set by None, which will be used to not add the current working domain to
    the parent CazymeProteinPrediction instance.
    
    :param domain_name: str, contains the CAZy (sub)family
    
    Return two strings, the CAZy family and the CAZy subfamily.
    """
    # check if unusal CAZy family format or subfamily was given
    if domain_name.find("_") != -1:
        try:
            re.match(r"\D{2,3}\d+?_\D", domain_name).group()  # check unusal CAZy family formating
            cazy_family = domain_name[:domain_name.find("_")]
            cazy_subfamily = np.nan

        except AttributeError:  # raised if not an usual CAZy family format
            try:
                # check if its a CAZy subfamily
                re.match(r"\D{2,3}\d+?_\d+", domain_name).group()
                cazy_family = domain_name[:domain_name.find("_")]
                cazy_subfamily = np.nan

            except AttributeError:  # raised if it doesn't match the expected CAZy subfamily format
                print(
                    f"WARNING -- UNKNOWN DATA TYPE OF {domain_name} FOR {protein_accession}\n"
                    "Not adding as a domain."
                )
                return None, None

    else:
        # potentially have a CAZy family
        try:
            re.match(r"\D{2,3}\d+", domain_name).group()
            cazy_family = domain_name
            cazy_subfamily = np.nan

        except AttributeError:  # raised if doesn't match expected CAZy family
            print(
                f"WARNING -- UNKNOWN DATA TYPE OF {domain_name} FOR {protein_accession}\n"
                "Not adding as a domain."
            )
            return None, None
    
    return cazy_family, cazy_subfamily


def get_hotpep_prediction(hotpep_data, protein_accession):
    """Retrieve Hotpep prediciton data from the dbCAN overview.txt file.
    
    :param hmmer_data: str, output data from Hotpep written in the overview.txt file.
    :param protein_accession: str
    
    Return a CazymeProteinPrediction instance.
    """
    # check if the protein was classified as a non-CAZyme
    if hotpep_data == "-":
        cazyme_classification = 0  # non-CAZyme
        protein = CazymeProteinPrediction("Hotpep", protein_accession, cazyme_classification)
        return protein
    
    # create a CazymeProteinPrediction instance to represent the CAZyme
    cazyme_classification = 1
    parent_cazyme = CazymeProteinPrediction("Hotpep", protein_accession, cazyme_classification)

    # Hotpep can predicted multiple CAZyme domaisn for a CAZyme
    # Hotpep separates each of the predicted CAZyme domains by additiona symbols
    parent_cazyme.cazyme_domains = get_hotpep_cazyme_domains(hotpep_data.split("+"), protein_accession)
    
    return parent_cazyme


def get_hotpep_cazyme_domains(predicted_domains, protein_accession):
    """Retrieve the CAZyme domains predicted by Hotpep.
    
    :param predicted_domains: list of strings, each string containing a CAZyme domain prediction.
    
    Return list of CazymeDomain instances. Each instance represents a unique CAZyme domain.
    """
    cazyme_domains = []
    
    for domain in predicted_domains:
        # remove k-mer group number
        domain = domain.split("(")[0]
        
        # check if a CAZy subfamily was predicted
        if domain.find("_") != -1:  # found a CAZy subfamily
            try:
                # check if its a CAZy subfamily
                re.match(r"\D{2,3}\d+?_\d+", domain).group()
                cazy_family = domain[:domain.find("_")]
                cazy_subfamily = np.nan

            except AttributeError:  # raised if it doesn't match the expected CAZy subfamily format
                print(
                    f"WARNING -- UNKNOWN DATA TYPE OF {domain} FOR {protein_accession}\n"
                    "Not adding as a domain."
                )
                continue
        
        else:  # found a CAZy family
            try:  # check it is a CAZy family
                re.match(r"\D{2,3}\d+", domain).group()
                cazy_family = domain
                cazy_subfamily = np.nan

            except AttributeError:  # raised if doesn't match expected CAZy family
                print(
                    f"WARNING -- UNKNOWN DATA TYPE OF {domain} FOR {protein_accession}\n"
                    "Not adding as a domain."
                )
                continue
        
        domain = CazymeDomain("Hotpep", protein_accession, cazy_family, cazy_subfamily)
        cazyme_domains.append(domain)
    
    return cazyme_domains
        

def get_hotpep_ec_numbers(overview_dict, hotpep_file):
    """Retrieve EC numbers from the Hotpep.out file, and add them to the CAZyme domains.
    
    :param overview_dict: dict, keyed by protein accession, valued by dictionary of 
                            predicion_tool:CazymeDomain
    :param hotpep_file: Path, path to Hotpep.out file
    
    Return overview_dict
    """
    with open (hotpep_file, "r") as fh:
        hotpep_file_data = fh.read().splitlines()
    
    for line in tqdm(hotpep_file_data[1:], desc="Parsing Hotpep.out"):  # skip the first line containing headings
        # separate out the data
        line = line.split("\t")
        
        protein_accession = line[2].strip()
        
        # retrieve the EC numbers
        ec_numbers = line[-1]
        ec_numbers = ec_numbers.split(",")

        # strip any remaining white space becuase the separating comma
        # is sometimes followed by a space
        index = 0
        for index in range(len(ec_numbers)):
            ec_numbers[index] = ec_numbers[index].strip()
            # remove k-mer group number (EC:kmer_group#)
            ec_numbers[index] = ec_numbers[index].split(":")[0]
        
            if ec_numbers[index] == "NA":
                ec_numbers[index] = np.nan
        
        # get CAZy family and subfamily of the current working domain
        cazy_family = line[0]
        if cazy_family.find("_") != -1:
            cazy_subfamily = cazy_family
            cazy_family = cazy_family[:cazy_family.find("_")]
        else:
            cazy_subfamily = np.nan

        domain_identifer = [cazy_family, cazy_subfamily]
        
        parent_cazyme = overview_dict[protein_accession]["Hotpep"]
        parent_domains = parent_cazyme.cazyme_domains
        
        for domain in parent_domains:
            if domain_identifer == [domain.cazy_family, domain.cazy_subfamily]:
                domain.ec_numbers = ec_numbers
        
        parent_cazyme.cazyme_domains = parent_domains
        overview_dict[protein_accession]["Hotpep"] = parent_cazyme
        
    return overview_dict


def get_diamond_prediction(diamond_data, protein_accession):
    """Retrieve DIAMOND prediction data from dbCAN overview.txt file.
    
    :param diamond_data, str, output data from DIAMOND written in the overview.txt
    :param protein_accession, str
    
    Return a CazymeProteinPrediction instance.
    """
    # check if the protein was classified as a non-CAZyme
    if diamond_data == "-":
        cazyme_classification = 0  # non-CAZyme
        protein = CazymeProteinPrediction("DIAMOND", protein_accession, cazyme_classification)
        return protein
    
    # create a CazymeProteinPrediction instance to represent the CAZyme
    cazyme_classification = 1
    parent_cazyme = CazymeProteinPrediction("DIAMOND", protein_accession, cazyme_classification)
    
    # DIAMOND can predict multiple CAZyme domains per CAZyme, each domain is separated by '+'
    parent_cazyme.cazyme_domains = get_diamond_predicted_domains(diamond_data.split("+"), protein_accession)
 
    return parent_cazyme


def get_diamond_predicted_domains(predicted_domains, protein_accession):
    """Retrieve the CAZyme domains predicted by DIAMOND, from the overview.txt file.
    
    :param predicted_domains: list of strings, each string contains a unique CAZyme domain prediction
    :param protein_accession: str
    
    Return list of CazymeDomain instances, one instance per unique CAZyme domain.
    """
    cazyme_domains = []  # store CazymeDomain instances
    
    for predicted_domain in predicted_domains:
        # check if a subfamily was predicted
        try:
            re.match(r"\D{2,3}\d+?_\d+", predicted_domain).group()
            cazy_family = predicted_domain[:predicted_domain.find("_")]
            cazy_subfamily = predicted_domain
        
        except AttributeError:  # raised if predicted_domain is not a CAZy subfamily
            try:
                # check if a CAZy family was predicted
                re.match(r"\D{2,3}\d+", predicted_domain).group()
                cazy_family = predicted_domain
                cazy_subfamily = np.nan
            
            except AttributeError:  # raised if not a CAZy family
                print(
                    f"WARNING -- Unexpected data formate for '{predicted_domain}' for\n"
                    "DIAMOND in the overview.txt file. Not adding domain to the CAZyme."
                )
            continue
        
        new_domain = CazymeDomain("DIAMOND", protein_accession, cazy_family, cazy_subfamily)
        cazyme_domains.append(new_domain)
    
    return cazyme_domains
    
    parent_cazyme.cazyme_domains = cazyme_domains


overview_path = "overview.txt"
hotpep_file = "Hotpep.out"
overview_dict = parse_dbcan_output(overview_path, hotpep_file)
print("protein count =", len(list(overview_dict.keys())))

multiple domain ranges= -CazymeDomain in ATY60367.1, fam=CBM66, subfam=nan- ['219-369', '388-547'] ATY60367.1
multiple domain ranges= -CazymeDomain in ATY60416.1, fam=GT2, subfam=nan- ['230-398', '375-584'] ATY60416.1
multiple domain ranges= -CazymeDomain in ATY63255.1, fam=GT57, subfam=nan- ['63-294', '333-565'] ATY63255.1
multiple domain ranges= -CazymeDomain in ATY66192.1, fam=GT2, subfam=nan- ['487-649', '627-867'] ATY66192.1




protein count = 445


<a id="dbcan_con"></a>

### Getting the dbCAN consensus

The column '#ofTools' in the overview.txt file gives the number of tools (out of HMMER, Hotpep and DIAMOND). Therefore, we can quickly check if the consensus of dbCAN is that a protein is not a CAZyme by checking if the value in the '#ofTools' column is 1. The consensus result of dbCAN is the result that at least two of the tools agree upon.

In [34]:
# Open the overview.txt file
with open("overview.txt", "r") as ofh:
    overview_file = ofh.read().splitlines()

# check if the consensus results is that the protein is a non-CAZyme
for line in tqdm(overview_file[1:], desc="Parsing overview.txt"): # skip the the first line in the file, becuase it contains the 'column' headings
    # separate the line content
    line = line.split("\t")

    protein_accession = line[0]
    print("#ofTools=", line[-1])

    if line[-1] == "1":
        # non-CAZyme
        cazyme_classification = 0  # non-CAZyme
        protein = CazymeProteinPrediction("dbCAN", protein_accession, cazyme_classification)
        print(protein)

#ofTools= 2
#ofTools= 3
#ofTools= 1
-CazymeProteinPrediction, protein=ATY58395.1, non-CAZyme-
#ofTools= 2
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 2
#ofTools= 3
#ofTools= 2
#ofTools= 3
#ofTools= 3
#ofTools= 2
#ofTools= 2
#ofTools= 1
-CazymeProteinPrediction, protein=ATY58653.1, non-CAZyme-
#ofTools= 3
#ofTools= 1
-CazymeProteinPrediction, protein=ATY58687.1, non-CAZyme-
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 1
-CazymeProteinPrediction, protein=ATY58910.1, non-CAZyme-
#ofTools= 1
-CazymeProteinPrediction, protein=ATY58915.1, non-CAZyme-
#ofTools= 3
#ofTools= 2
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 3
#ofTools= 2
#ofTools= 3
#ofTools= 3
#ofTools= 1
-CazymeProteinPrediction, protein=ATY59524.1, non-CAZyme-
#ofTools= 1
-CazymeProteinPrediction, protein=ATY59596.1, non-CAZyme-
#ofTools= 3
#ofTools= 1
-CazymeProteinPrediction, protein=ATY59663.1, non-CAZyme-
#ofTools= 2
#ofTools

So the code handles proteins with the consensus prediction of non-CAZyme, but it does not get the consensus CAZyme domain (CAZy family and subfamily) prediction for each protein predicted to be a CAZyme.

To save having to reparse the HMMER, Hotpep and DIAMOND data from current working like from the overview.txt file (remebering each line contains a unique protein, therefore, the current working line == the current working protein), we can parse the `CazymeProteinPrediction` instances representing the protein prediction each for HMMER, Hotpep and DIAMOND.

Each `CazymeProteinPrediciton` instance contains a list of `CazymeDomain` instances, which each contain a CAZy family and subfamily. We can create a `set()` of strings, with each string representing a unique `CazymeDomain` instance (`CazyFam_CazySubfam`). This is because sets cannot contain duplicate entries, therefore, we can use this to check for duplicate entries between the predicted CAZyme domains. Additionally, the `&` operator can be used to find only those elements between multiple sets that are located in all sets.

Using the `&` between three sets will find elements that are present in all three lists. Lets see a practical example. We can create three lists, `a`, `b` and `c`. '4' is common to all 3 lists, but '1' and '8' are common to only 2 of the 3 lists each.

In [35]:
a = [1,2,3,4]
b = [1,4,8,9]
c = [4,7,6,8]
consensus_a_b_c = list(set(a) & set(b) & set(c))
consensus_a_b_c

[4]

Above we can see that we have the element, '4', which is common to all 3 lists: `a`, `b` and `c`.

However, we are defining the consensus result of dbCaN being any result that at least two of the prediction tools agree upon. Therefore, we need to compare each of the pairs of prediction tools to find elements that are predicted by at least 2 of the tools. This is the same as comparing:
- `a` and `b`
- `b` and `c`
- `a` and `c`

In [36]:
a = [1,2,3,4]
b = [1,4,8,9]
c = [4,7,6,8]
consensus_a_b_c = list(set(a) & set(b) & set(c))
consensus_a_b = list(set(a) & set(b))
consensus_b_c = list(set(b) & set(c))
consensus_a_c = list(set(a) & set(c))
print("elements in all three lists =", consensus_a_b_c)
print("elements in a and b =", consensus_a_b)
print("elements in b and c =", consensus_b_c)
print("elements in a and c =", consensus_a_c)

elements in all three lists = [4]
elements in a and b = [1, 4]
elements in b and c = [8, 4]
elements in a and c = [4]


Each CAZyme domain in the list of CAZyme domains needs to be a string. If a CAZyme domain was represented by a list (`[cazy_family, cazy_subfamily]`) would raise a `TypeError: unhashable type: 'list'` when trying to create a set from a list of CAZyme domains.

In [37]:
def parse_dbcan_output(overview_path, hotpep_file):
    """Parse the data in the 'overview.txt' files from dbCAN.
    
    The 'overview.txt' file contains the CAZyme predicion data from
    HMMER, Hotpep and DIAMOND. A dbCAN consensus result is retrieved
    by finding predictions that at least to of the prediction tools
    agree upon.
    
    Each protein from the input FASTA file containing all query protein
    sequences parsed by dbCAN is represented by a CazymeProteinprediction
    instance. Each predicted CAZyme domains in each predicted CAZyme is
    represented by a CazymeDomain instance, which are sotred within their
    parent CAZyme's CazymeProteinPrediction instance. A CazymeProteinPrediction
    instance is created for each protein in the inpu FASTA file parsed
    by dbCAN and for each CAZyme prediction tool: dbCAN, HMMER, Hotpep and
    DIAMOND.
    
    :param overview_path: Path, path to dbCAN output overview.txt file
    :param hotpep_file: Path, path to Hotpep.out file
    
    Return 4 lists, each containing the CazymeProteinPrediction instances for
    HMMER, Hotpep, DIAMOND and dbCAN.
    """
    # open the overview.txt file
    with open(overview_path, "r") as ofh:
        overview_file = ofh.read().splitlines()
    
    # create an empty dictionary to store the CazymeProteinPrediction instances
    # {protein_accesion: CazymeProteinPrediction_instance}
    overview_dict = {}

    # skip the the first line in the file, becuase it contains the 'column' headings
    for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):
        # separate the line content
        line = line.split("\t")
        
        protein_accession = line[0]
        
        # create a CazymeProteinPrediction instance each for HMMER, Hotpep and DIAMOND
        hmmer_prediction = get_hmmer_prediction(line[1], protein_accession)
        hotpep_prediction = get_hotpep_prediction(line[2], protein_accession)
        diamond_prediction = get_diamond_prediction(line[3], protein_accession)
        
        # get the dbCAN consensus result
        dbcan_prediction = get_dbcan_consensus(hmmer_prediction, hotpep_prediction, diamond_prediction, protein_accession, line[-1])
        
        # add the CazymeProteinPrediction instances to the overview_dict
        try:
            overview_dict[protein_accession]
            prediction_outputs = [["HMMER", hmmer_prediction], ["Hotpep", hotpep_prediction], ["DIAMOND", diamond_prediction], ["dbCAN", dbcan_prediction]]
            
            for prediction_tool in prediction_outputs:
                try:
                    overview_dict[protein_accession][prediction_tool[0]]
                    print(
                        f"WARNING -- Protein {protein_accession} already catalogued with {prediction_tool[0]}"
                    )
                except KeyError:  # raised if CazymeProteinPrediciton instance not stored for prediction_tool
                    overview_dict[protein_accession][prediction_tool[0]] = prediction_tool[1]
        
        except KeyError:  # raised if protein has not been added yet
            overview_dict[protein_accession] = {
                "HMMER": hmmer_prediction,
                "Hotpep": hotpep_prediction,
                "DIAMOND": diamond_prediction,
                "dbCAN": "dbcan_prediction",
            }
    
    # get EC numbers predicted by Hotpep
    overview_dict = get_hotpep_ec_numbers(overview_dict, hotpep_file)
    return overview_dict
        

def get_dbcan_consensus(hmmer_prediction, hotpep_prediction, diamond_prediction, protein_prediction, no_of_tools):
    """Retrieve the consensus CAZyme and CAZyme domain prediction for dbCAN.
    
    Consensus is defined as a result that at least two of the tools (HMMER, Hotpep and DIAMOND)
    agree upon.
    
    :param hmmer_prediction: CazymeProteinPrediction instance containing the HMMER prediction
    :param hotpep_prediction: CazymeProteinPrediction instance containing the Hotpep prediction
    :param diamond_prediction: CazymeProteinPrediction instance containing the DIAMOND prediction
    :param protein_accession: str
    :param no_of_tools: str, data from the '#ofTools' column from the overview.txt file, it states
        the number of tools that list the protein as a CAZyme
    
    Return CazymeProteinPrediction instance to represent the dbCAN consensus prediction
    """
    # first check if the consensus was that the protein was a non-CAZyme
    if no_of_tools == "1":
        # non-CAZyme
        cazyme_classification = 0  # non-CAZyme
        protein = CazymeProteinPrediction("dbCAN", protein_accession, cazyme_classification)
        print("non_cazyme= ", protein)
        return protein
    
    # get the consensus CAZyme domain predicitons
    
    # first get each of the predicted CAZyme domains for each tool
    print(protein_accession)
    hmmer_domains = get_dbcan_cazyme_domains(hmmer_prediction)
    print("hmmer=", hmmer_domains)
    hotpep_domains = get_dbcan_cazyme_domains(hotpep_prediction)
    print("hotpep=", hotpep_domains)
    diamond_domains = get_dbcan_cazyme_domains(diamond_prediction)
    print("diamond=", diamond_domains)


def get_dbcan_cazyme_domains(cazyme_prediction):
    """Retrieve the predicted CAZyme domains, representing each domains as a string 'fam_subfam'.
    
    :param cazyme_prediction: CazymeProteinPrediction instance.
    
    Return a list of strings with each string representing a single predicted CAZyme domain.
    """
    cazyme_domain_instances = cazyme_prediction.cazyme_domains
    
    cazyme_domains = []  # store list representing CAZyme domains [fam(str), subfam(str)]
    
    # if a prediction tool predicts a protein is a non_CAZyme, non_cazymes == np.nan (a float)
    if type(cazyme_domain_instances) is not float:
        for domain in cazyme_domain_instances:
            cazy_family = domain.cazy_family
            cazy_subfamily = domain.cazy_subfamily
            cazyme_domains.append(f"{cazy_family}_{cazy_subfamily}")
    
    return cazyme_domains


overview_path = "overview.txt"
hotpep_file = "Hotpep.out"
overview_dict = parse_dbcan_output(overview_path, hotpep_file)
print("protein count =", len(list(overview_dict.keys())))

ATY62977.1
hmmer= ['CE5_nan']
hotpep= []
diamond= []
ATY62977.1
hmmer= ['AA11_nan']
hotpep= ['AA11_nan']
diamond= []
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
ATY62977.1
hmmer= ['AA6_nan']
hotpep= ['AA6_nan']
diamond= []
ATY62977.1
hmmer= ['AA3_nan']
hotpep= ['AA3_nan']
diamond= []
ATY62977.1
hmmer= ['GH16_nan']
hotpep= ['GH16_nan']
diamond= []
ATY62977.1
hmmer= ['GH18_nan']
hotpep= ['GH18_nan', 'CBM1_nan', 'CBM19_nan']
diamond= []
ATY62977.1
hmmer= ['GH5_nan']
hotpep= ['GH5_nan']
diamond= ['GH5_GH5_24']
ATY62977.1
hmmer= ['GH55_nan']
hotpep= ['GH55_nan']
diamond= []
ATY62977.1
hmmer= ['GH16_nan']
hotpep= []
diamond= []
ATY62977.1
hmmer= ['GH64_nan']
hotpep= ['GH64_nan', 'CBM6_nan']
diamond= []
ATY62977.1
hmmer= ['CE5_nan']
hotpep= []
diamond= []
ATY62977.1
hmmer= ['GH76_nan']
hotpep= ['GH76_nan']
diamond= []
ATY62977.1
hmmer= ['GH38_nan']
hotpep= ['GH38_nan']
diamond= []
ATY62977.1
hmmer= ['GT2_nan']
hotpep= []
diamond= []
ATY62977.1
hmmer= ['GH12_nan']
ho

diamond= []
ATY62977.1
hmmer= ['GT15_nan']
hotpep= ['GT15_nan']
diamond= []
ATY62977.1
hmmer= ['GH18_nan']
hotpep= []
diamond= []
ATY62977.1
hmmer= ['GH72_nan']
hotpep= ['GH72_nan', 'CBM43_nan']
diamond= []
ATY62977.1
hmmer= ['GH2_nan']
hotpep= ['GH2_nan']
diamond= []
ATY62977.1
hmmer= ['GH16_nan']
hotpep= []
diamond= []
ATY62977.1
hmmer= ['GT22_nan']
hotpep= ['GT22_nan']
diamond= []
ATY62977.1
hmmer= ['GH13_nan']
hotpep= ['GH13_nan']
diamond= ['GH13_GH13_40']
ATY62977.1
hmmer= ['GT39_nan']
hotpep= ['GT39_nan']
diamond= []
ATY62977.1
hmmer= ['GH47_nan']
hotpep= ['GH47_nan']
diamond= []
ATY62977.1
hmmer= ['GH5_nan']
hotpep= ['GH5_nan']
diamond= ['GH5_GH5_11']
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
ATY62977.1
hmmer= ['GT32_nan']
hotpep= []
diamond= []
ATY62977.1
hmmer= ['GH132_nan']
hotpep= ['GH132_nan']
diamond= []
ATY62977.1
hmmer= ['GT22_nan']
hotpep= ['GT22_nan']
diamond= []
ATY62977.1
hmmer= ['GH5_nan']
hotpep= ['GH5_nan']
diamond= ['GH5_GH5_5']
ATY62

diamond= []
ATY62977.1
hmmer= ['AA7_nan']
hotpep= ['AA7_nan', 'CBM18_nan']
diamond= []
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
ATY62977.1
hmmer= ['CBM21_nan']
hotpep= ['CBM21_nan']
diamond= []
ATY62977.1
hmmer= ['GT2_nan']
hotpep= []
diamond= []
ATY62977.1
hmmer= ['AA1_nan']
hotpep= ['AA1_nan']
diamond= ['AA1_AA1_3']
ATY62977.1
hmmer= ['CE5_nan']
hotpep= ['CE5_nan']
diamond= []
ATY62977.1
hmmer= ['CE16_nan']
hotpep= ['CE16_nan']
diamond= []
ATY62977.1
hmmer= ['GH47_nan']
hotpep= ['GH47_nan']
diamond= []
ATY62977.1
hmmer= ['GH17_nan']
hotpep= ['GH17_nan']
diamond= []
ATY62977.1
hmmer= ['GH16_nan']
hotpep= []
diamond= []
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
ATY62977.1
hmmer= ['GH128_nan']
hotpep= ['GH128_nan']
diamond= []
ATY62977.1
hmmer= ['GH79_nan']
hotpep= []
diamond= []
ATY62977.1
hmmer= ['GH18_nan']
hotpep= ['GH18_nan', 'CBM18_nan']
diamond= []
ATY62977.1
hmmer= ['GH76_nan']
hotpep= ['GH76_nan']
diamond= []
ATY62977.1

non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_ca


protein count = 445


From the printed statements above we can see the method retrieves a list of strings, with each string representing a unique CAZyme domain.

Now we can add a method to find the consensus (where at least 2 prediction tools agree) between each od the predicted CAZyme domains.

In [38]:
def parse_dbcan_output(overview_path, hotpep_file):
    """Parse the data in the 'overview.txt' files from dbCAN.
    
    The 'overview.txt' file contains the CAZyme predicion data from
    HMMER, Hotpep and DIAMOND. A dbCAN consensus result is retrieved
    by finding predictions that at least to of the prediction tools
    agree upon.
    
    Each protein from the input FASTA file containing all query protein
    sequences parsed by dbCAN is represented by a CazymeProteinprediction
    instance. Each predicted CAZyme domains in each predicted CAZyme is
    represented by a CazymeDomain instance, which are sotred within their
    parent CAZyme's CazymeProteinPrediction instance. A CazymeProteinPrediction
    instance is created for each protein in the inpu FASTA file parsed
    by dbCAN and for each CAZyme prediction tool: dbCAN, HMMER, Hotpep and
    DIAMOND.
    
    :param overview_path: Path, path to dbCAN output overview.txt file
    :param hotpep_file: Path, path to Hotpep.out file
    
    Return 4 lists, each containing the CazymeProteinPrediction instances for
    HMMER, Hotpep, DIAMOND and dbCAN.
    """
    # open the overview.txt file
    with open(overview_path, "r") as ofh:
        overview_file = ofh.read().splitlines()
    
    # create an empty dictionary to store the CazymeProteinPrediction instances
    # {protein_accesion: CazymeProteinPrediction_instance}
    overview_dict = {}

    # skip the the first line in the file, becuase it contains the 'column' headings
    for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):
        # separate the line content
        line = line.split("\t")
        
        protein_accession = line[0]
        
        # create a CazymeProteinPrediction instance each for HMMER, Hotpep and DIAMOND
        hmmer_prediction = get_hmmer_prediction(line[1], protein_accession)
        hotpep_prediction = get_hotpep_prediction(line[2], protein_accession)
        diamond_prediction = get_diamond_prediction(line[3], protein_accession)
        
        # get the dbCAN consensus result
        dbcan_prediction = get_dbcan_consensus(hmmer_prediction, hotpep_prediction, diamond_prediction, protein_accession, line[-1])
        
        # add the CazymeProteinPrediction instances to the overview_dict
        try:
            overview_dict[protein_accession]
            prediction_outputs = [["HMMER", hmmer_prediction], ["Hotpep", hotpep_prediction], ["DIAMOND", diamond_prediction], ["dbCAN", dbcan_prediction]]
            
            for prediction_tool in prediction_outputs:
                try:
                    overview_dict[protein_accession][prediction_tool[0]]
                    print(
                        f"WARNING -- Protein {protein_accession} already catalogued with {prediction_tool[0]}"
                    )
                except KeyError:  # raised if CazymeProteinPrediciton instance not stored for prediction_tool
                    overview_dict[protein_accession][prediction_tool[0]] = prediction_tool[1]
        
        except KeyError:  # raised if protein has not been added yet
            overview_dict[protein_accession] = {
                "HMMER": hmmer_prediction,
                "Hotpep": hotpep_prediction,
                "DIAMOND": diamond_prediction,
                "dbCAN": dbcan_prediction,
            }
    
    # get EC numbers predicted by Hotpep
    overview_dict = get_hotpep_ec_numbers(overview_dict, hotpep_file)
    return overview_dict
        

def get_dbcan_consensus(hmmer_prediction, hotpep_prediction, diamond_prediction, protein_prediction, no_of_tools):
    """Retrieve the consensus CAZyme and CAZyme domain prediction for dbCAN.
    
    Consensus is defined as a result that at least two of the tools (HMMER, Hotpep and DIAMOND)
    agree upon.
    
    :param hmmer_prediction: CazymeProteinPrediction instance containing the HMMER prediction
    :param hotpep_prediction: CazymeProteinPrediction instance containing the Hotpep prediction
    :param diamond_prediction: CazymeProteinPrediction instance containing the DIAMOND prediction
    :param protein_accession: str
    :param no_of_tools: str, data from the '#ofTools' column from the overview.txt file, it states
        the number of tools that list the protein as a CAZyme
    
    Return CazymeProteinPrediction instance to represent the dbCAN consensus prediction
    """
    # first check if the consensus was that the protein was a non-CAZyme
    if no_of_tools == "1":
        # non-CAZyme
        cazyme_classification = 0  # non-CAZyme
        protein = CazymeProteinPrediction("dbCAN", protein_accession, cazyme_classification)
        print("non_cazyme= ", protein)
        return protein
    
    # get the consensus CAZyme domain predicitons
    
    # first get each of the predicted CAZyme domains for each tool
    hmmer_domains = get_cazyme_domains(hmmer_prediction)
    hotpep_domains = get_cazyme_domains(hotpep_prediction)
    diamond_domains = get_cazyme_domains(diamond_prediction)
    
    print("PROTEIN_ACCESSION=", protein_accession)
    # get the predicted CAZyme domains that are predicted by at two tools
    hmmer_hotpep_consensus = list(set(hmmer_domains) & set(hotpep_domains))
    hotpep_diamond_consensus = list(set(hotpep_domains) & set(diamond_domains))
    hmmer_diamond_consensus = list(set(hmmer_domains) & set(diamond_domains))
    print("hmmer=", hmmer_domains)
    print("hotpep=", hotpep_domains)
    print("diamond=", diamond_domains)
    print("hmmer-hotpep=", hmmer_hotpep_consensus)
    print("hotpep_diamond=", hotpep_diamond_consensus)
    print("hmmer_diamond=", hmmer_diamond_consensus)

    # get the CAZyme domains predicted by all prediction tools
    all_consensus = list(set(hmmer_domains) & set(hotpep_domains) & set(diamond_domains))
    print("all=", all_consensus)

    # combine all consensus results
    combined_consensus = (
        hmmer_hotpep_consensus + hotpep_diamond_consensus + hmmer_diamond_consensus + all_consensus
    )
    # remove duplicate entries
    combined_consensus = list(set(combined_consensus))
    
    # create a CazymeProteinPrediction instance to represent the CAZyme
    cazyme_classification = 1  # CAZyme
    parent_cazyme = CazymeProteinPrediction("dbCAN", protein_accession, cazyme_classification)
    
    # create a CazymeDomain instance for each domain
    cazyme_domains = []  # store CazymeDomain instances
    for domain in combined_consensus:
        domain = domain.split("**")
        cazy_family = domain[0]

        if domain[1] == "nan":
            cazy_subfamily = np.nan
        else:
            cazy_subfamily = domain[1]

        new_domain = CazymeDomain("dbCAN", protein_accession, cazy_family, cazy_subfamily)

        cazyme_domains.append(new_domain)
    
    # add predicted CAZyme domains to the instance representing the CAZyme
    parent_cazyme.cazyme_domains = cazyme_domains
    print("domains=", cazyme_domains)
    print("========================================================================")
    
    return parent_cazyme
    
    
def get_cazyme_domains(cazyme_prediction):
    """Retrieve the predicted CAZyme domains, representing each domains as a string 'fam_subfam'.
    
    :param cazyme_prediction: CazymeProteinPrediction instance.
    
    Return a list of strings with each string representing a single predicted CAZyme domain.
    """
    cazyme_domain_instances = cazyme_prediction.cazyme_domains
    
    cazyme_domains = []  # store list representing CAZyme domains [fam(str), subfam(str)]
    
    # if a prediction tool predicts a protein is a non_CAZyme, non_cazymes == np.nan (a float)
    if type(cazyme_domain_instances) is not float:
        for domain in cazyme_domain_instances:
            cazy_family = domain.cazy_family
            cazy_subfamily = domain.cazy_subfamily
            # separate the family and subfamily '**' becuase these do not appear in the either
            cazyme_domains.append(f"{cazy_family}**{cazy_subfamily}")
    
    return cazyme_domains


overview_path = "overview.txt"
hotpep_file = "Hotpep.out"
overview_dict = parse_dbcan_output(overview_path, hotpep_file)
print("protein count =", len(list(overview_dict.keys())))

PROTEIN_ACCESSION= ATY62977.1
hmmer= ['CE5**nan']
hotpep= []
diamond= []
hmmer-hotpep= []
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= []
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['AA11**nan']
hotpep= ['AA11**nan']
diamond= []
hmmer-hotpep= ['AA11**nan']
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= [<CazymeDomain parent=ATY62977.1 fam=AA11, subfam=nan>]
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['AA6**nan']
hotpep= ['AA6**nan']
diamond= []
hmmer-hotpep= ['AA6**nan']
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= [<CazymeDomain parent=ATY62977.1 fam=AA6, subfam=nan>]
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['AA3**nan']
hotpep= ['AA3**nan']
diamond= []
hmmer-hotpep= ['AA3**nan']
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= [<CazymeDomain parent=ATY62977.1 fam=AA3, subfam=nan>]
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['GH16**nan']
hotpep= ['GH16**nan']
diamond= []
hmmer-hotpep= ['GH16**nan']
hotpep_d

hmmer_diamond= []
all= []
domains= [<CazymeDomain parent=ATY62977.1 fam=GH16, subfam=nan>]
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['GH16**nan']
hotpep= ['GH16**nan', 'CBM4**nan', 'CBM54**nan', 'CBM6**nan']
diamond= []
hmmer-hotpep= ['GH16**nan']
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= [<CazymeDomain parent=ATY62977.1 fam=GH16, subfam=nan>]
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['GT2**nan']
hotpep= ['GT0**nan']
diamond= []
hmmer-hotpep= []
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= []
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['GH15**nan']
hotpep= ['GH15**nan']
diamond= []
hmmer-hotpep= ['GH15**nan']
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= [<CazymeDomain parent=ATY62977.1 fam=GH15, subfam=nan>]
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['GH18**nan']
hotpep= ['GH18**nan', 'CBM18**nan']
diamond= []
hmmer-hotpep= ['GH18**nan']
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= [<CazymeDomain parent=ATY62977.1 fam=GH18, subfam=nan>]
PROTEIN_ACCESSION= ATY62977.1

all= []
domains= []
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['GT17**nan']
hotpep= ['GT17**nan']
diamond= []
hmmer-hotpep= ['GT17**nan']
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= [<CazymeDomain parent=ATY62977.1 fam=GT17, subfam=nan>]
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['GH72**nan']
hotpep= ['GH72**nan', 'CBM43**nan']
diamond= []
hmmer-hotpep= ['GH72**nan']
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= [<CazymeDomain parent=ATY62977.1 fam=GH72, subfam=nan>]
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['PL8**nan']
hotpep= []
diamond= ['PL8**PL8_4']
hmmer-hotpep= []
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= []
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['GT57**nan']
hotpep= ['GT57**nan']
diamond= []
hmmer-hotpep= ['GT57**nan']
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= [<CazymeDomain parent=ATY62977.1 fam=GT57, subfam=nan>]
PROTEIN_ACCESSION= ATY62977.1
hmmer= ['GT4**nan']
hotpep= []
diamond= []
hmmer-hotpep= []
hotpep_diamond= []
hmmer_diamond= []
all= []
dom

PROTEIN_ACCESSION= ATY62977.1
hmmer= []
hotpep= ['GT31**nan']
diamond= []
hmmer-hotpep= []
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= []
PROTEIN_ACCESSION= ATY62977.1
hmmer= []
hotpep= ['GT31**nan']
diamond= []
hmmer-hotpep= []
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= []
PROTEIN_ACCESSION= ATY62977.1
hmmer= []
hotpep= ['GT31**nan']
diamond= []
hmmer-hotpep= []
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= []
PROTEIN_ACCESSION= ATY62977.1
hmmer= []
hotpep= ['GT31**nan']
diamond= []
hmmer-hotpep= []
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= []
PROTEIN_ACCESSION= ATY62977.1
hmmer= []
hotpep= ['GT41**nan']
diamond= []
hmmer-hotpep= []
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= []
PROTEIN_ACCESSION= ATY62977.1
hmmer= []
hotpep= ['CBM10**nan', 'CBM2**nan']
diamond= []
hmmer-hotpep= []
hotpep_diamond= []
hmmer_diamond= []
all= []
domains= []
non_cazyme=  -CazymeProteinPrediction, protein=ATY62977.1, non-CAZyme-
non_cazyme=  -CazymeProtein


protein count = 445


We can see from the printed statements that we know generate a consensus result for dbCAN, representing all data that at least two of the three prediction tools (HMMER, Hotpep and DIAMOND). Additionally, this creates a `CazymeProteinPrediction` instance to represent a CAZyme, which is populated with a `CazymeDomain` instance for each consensus CAZyme domain prediction.

<a id="non-caz"></a>

## Get non-CAZymes

The proteins from the input FASTA (which contains all query protein sequences parsed by dbCAN) that were classified as a non-CAZyme are not included in any of the output files from dbCAN and its contained prediction tools. However, it is useful to have a complete dataset of the all CAZyme/non-CAZyme predictions from the input FASTA file of query sequences. Additionally, for the evaluation of the prediction tools we need to know specifically, which proteins were classified as CAZymes and which were classified as non-CAZymes.

Therefore, the final task is to parse the input FASTA file of query protein sequences and create `CazymeProteinPrediction` instances to represent the non-CAZymes predicted by dbCAN.

The dictionary storing the `CazymeProteinPrediction` instances generated by parsing the 'overview.txt' file is keyed by the protein accessions retrieved from the 'overview.txt' file. These 'protein_accessions' are identical to the protein data line (the line prefixed with '>'). Therefore, we can parse the input FASTA file of query protein sequences and check if the protein data from the line prefixed with '>' is a key in our dictionary of `CazymeProteinPrediction` instances. If it is not a key then we can add a new key with a non-CAZyme prediciton for HMMER, Hotpep, DIAMOND and dbCAN.

In [55]:
def parse_dbcan_output(overview_path, hotpep_file, fasta_file):
    """Parse the data in the 'overview.txt' files from dbCAN.
    
    The 'overview.txt' file contains the CAZyme predicion data from
    HMMER, Hotpep and DIAMOND. A dbCAN consensus result is retrieved
    by finding predictions that at least to of the prediction tools
    agree upon.
    
    Each protein from the input FASTA file containing all query protein
    sequences parsed by dbCAN is represented by a CazymeProteinprediction
    instance. Each predicted CAZyme domains in each predicted CAZyme is
    represented by a CazymeDomain instance, which are sotred within their
    parent CAZyme's CazymeProteinPrediction instance. A CazymeProteinPrediction
    instance is created for each protein in the inpu FASTA file parsed
    by dbCAN and for each CAZyme prediction tool: dbCAN, HMMER, Hotpep and
    DIAMOND.
    
    :param overview_path: Path, path to dbCAN output overview.txt file
    :param hotpep_file: Path, path to Hotpep.out file
    :param fasta_file: Path, path to the input FASTA file containing the query sequences
        parsed by dbCAN, HMMER, Hotpep and DIAMOND
    
    Return 4 lists, each containing the CazymeProteinPrediction instances for
    HMMER, Hotpep, DIAMOND and dbCAN.
    """
    # open the overview.txt file
    with open(overview_path, "r") as ofh:
        overview_file = ofh.read().splitlines()
    
    # create an empty dictionary to store the CazymeProteinPrediction instances
    # {protein_accesion: CazymeProteinPrediction_instance}
    overview_dict = {}

    # skip the the first line in the file, becuase it contains the 'column' headings
    for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):
        # separate the line content
        line = line.split("\t")
        
        protein_accession = line[0].strip()
        
        # create a CazymeProteinPrediction instance each for HMMER, Hotpep and DIAMOND
        hmmer_prediction = get_hmmer_prediction(line[1], protein_accession)
        hotpep_prediction = get_hotpep_prediction(line[2], protein_accession)
        diamond_prediction = get_diamond_prediction(line[3], protein_accession)
        
        # get the dbCAN consensus result
        dbcan_prediction = get_dbcan_consensus(hmmer_prediction, hotpep_prediction, diamond_prediction, protein_accession, line[-1])
        
        # add the CazymeProteinPrediction instances to the overview_dict
        try:
            overview_dict[protein_accession]
            prediction_outputs = [["HMMER", hmmer_prediction], ["Hotpep", hotpep_prediction], ["DIAMOND", diamond_prediction], ["dbCAN", dbcan_prediction]]
            
            for prediction_tool in prediction_outputs:
                try:
                    overview_dict[protein_accession][prediction_tool[0]]
                    print(
                        f"WARNING -- Protein {protein_accession} already catalogued with {prediction_tool[0]}"
                    )
                except KeyError:  # raised if CazymeProteinPrediciton instance not stored for prediction_tool
                    overview_dict[protein_accession][prediction_tool[0]] = prediction_tool[1]
        
        except KeyError:  # raised if protein has not been added yet
            overview_dict[protein_accession] = {
                "HMMER": hmmer_prediction,
                "Hotpep": hotpep_prediction,
                "DIAMOND": diamond_prediction,
                "dbCAN": dbcan_prediction,
            }
    
    # get EC numbers predicted by Hotpep
    # overview_dict = get_hotpep_ec_numbers(overview_dict, hotpep_file)
    
    # get non-CAZymes
    overview_dict = get_non_cazymes(fasta_file, overview_dict)
    return overview_dict
        

def get_dbcan_consensus(hmmer_prediction, hotpep_prediction, diamond_prediction, protein_prediction, no_of_tools):
    """Retrieve the consensus CAZyme and CAZyme domain prediction for dbCAN.
    
    Consensus is defined as a result that at least two of the tools (HMMER, Hotpep and DIAMOND)
    agree upon.
    
    :param hmmer_prediction: CazymeProteinPrediction instance containing the HMMER prediction
    :param hotpep_prediction: CazymeProteinPrediction instance containing the Hotpep prediction
    :param diamond_prediction: CazymeProteinPrediction instance containing the DIAMOND prediction
    :param protein_accession: str
    :param no_of_tools: str, data from the '#ofTools' column from the overview.txt file, it states
        the number of tools that list the protein as a CAZyme
    
    Return CazymeProteinPrediction instance to represent the dbCAN consensus prediction
    """
    # first check if the consensus was that the protein was a non-CAZyme
    if no_of_tools == "1":
        # non-CAZyme
        cazyme_classification = 0  # non-CAZyme
        protein = CazymeProteinPrediction("dbCAN", protein_accession, cazyme_classification)
        return protein
    
    # get the consensus CAZyme domain predicitons
    
    # first get each of the predicted CAZyme domains for each tool
    hmmer_domains = get_cazyme_domains(hmmer_prediction)
    hotpep_domains = get_cazyme_domains(hotpep_prediction)
    diamond_domains = get_cazyme_domains(diamond_prediction)
    
    # get the predicted CAZyme domains that are predicted by at two tools
    hmmer_hotpep_consensus = list(set(hmmer_domains) & set(hotpep_domains))
    hotpep_diamond_consensus = list(set(hotpep_domains) & set(diamond_domains))
    hmmer_diamond_consensus = list(set(hmmer_domains) & set(diamond_domains))

    # get the CAZyme domains predicted by all prediction tools
    all_consensus = list(set(hmmer_domains) & set(hotpep_domains) & set(diamond_domains))

    # combine all consensus results
    combined_consensus = (
        hmmer_hotpep_consensus + hotpep_diamond_consensus + hmmer_diamond_consensus + all_consensus
    )
    # remove duplicate entries
    combined_consensus = list(set(combined_consensus))
    
    # create a CazymeProteinPrediction instance to represent the CAZyme
    cazyme_classification = 1  # CAZyme
    parent_cazyme = CazymeProteinPrediction("dbCAN", protein_accession, cazyme_classification)
    
    # create a CazymeDomain instance for each domain
    cazyme_domains = []  # store CazymeDomain instances
    for domain in combined_consensus:
        domain = domain.split("**")
        cazy_family = domain[0]

        if domain[1] == "nan":
            cazy_subfamily = np.nan
        else:
            cazy_subfamily = domain[1]

        new_domain = CazymeDomain("dbCAN", protein_accession, cazy_family, cazy_subfamily)

        cazyme_domains.append(new_domain)
    
    # add predicted CAZyme domains to the instance representing the CAZyme
    parent_cazyme.cazyme_domains = cazyme_domains
    
    return parent_cazyme
    
    
def get_cazyme_domains(cazyme_prediction):
    """Retrieve the predicted CAZyme domains, representing each domains as a string 'fam_subfam'.
    
    :param cazyme_prediction: CazymeProteinPrediction instance.
    
    Return a list of strings with each string representing a single predicted CAZyme domain.
    """
    cazyme_domain_instances = cazyme_prediction.cazyme_domains
    
    cazyme_domains = []  # store list representing CAZyme domains [fam(str), subfam(str)]
    
    # if a prediction tool predicts a protein is a non_CAZyme, non_cazymes == np.nan (a float)
    if type(cazyme_domain_instances) is not float:
        for domain in cazyme_domain_instances:
            cazy_family = domain.cazy_family
            cazy_subfamily = domain.cazy_subfamily
            # separate the family and subfamily '**' becuase these do not appear in the either
            cazyme_domains.append(f"{cazy_family}**{cazy_subfamily}")
    
    return cazyme_domains


def get_non_cazymes(fasta_path, overview_dict):
    """Add proteins that dbCAN identified as non-CAZymes to the collection of CazymeProteinPrediction instances.
    
    :param fasta_path: Path, FASTA file used as input for CUPP.
    :param overview_dict: dict, key=protein_accession, value=dict valued by prediction tool,
          valued by CazymeProteinPrediction intance
    
    Return a dictionary keyed by protein accessions and valued by dictionary of tool:CazymeProteinPrediction.
    """
    print("len PRE adding non-CAZyme=", len(list(overview_dict.values())))
    # open the FASTA path
    with open(fasta_path) as fh:
        fasta = fh.read().splitlines()
    
    fasta_protein_total = 0

    for line in tqdm(fasta, desc="Adding non-CAZymes"):
        if line.startswith(">"):
            fasta_protein_total += 1
            protein_accession = line[1:].split(" ")[0].strip()
            
            # check if the protein has been listed as a CAZyme by CUPP
            try:
                overview_dict[protein_accession]
            except KeyError:
                # raised if protein not in cupp_predictions, inferring proten was not labelled as CAZyme
                cazyme_classification = 0
                overview_dict[protein_accession] = {}
                for prediction_tool in ["HMMER", "Hotpep", "DIAMOND", "dbCAN"]:
                    overview_dict[protein_accession][prediction_tool] = CazymeProteinPrediction(
                        prediction_tool,
                        protein_accession,
                        cazyme_classification,
                    )
    print("len AFTER adding non-CAZyme=", len(list(overview_dict.values())))
    print("number of proteins in the input FASTA file of query protien sequences=", fasta_protein_total)
    return overview_dict

overview_path = "overview.txt"
hotpep_file = "Hotpep.out"
fasta_file = "genbank_proteins_txid73501_GCA_008080495_1.fasta"
overview_dict = parse_dbcan_output(overview_path, hotpep_file, fasta_file)
print("protein count =", len(list(overview_dict.keys())))

multiple domain ranges= -CazymeDomain in ATY60367.1, fam=CBM66, subfam=nan- ['219-369', '388-547'] ATY60367.1
multiple domain ranges= -CazymeDomain in ATY60416.1, fam=GT2, subfam=nan- ['230-398', '375-584'] ATY60416.1
multiple domain ranges= -CazymeDomain in ATY63255.1, fam=GT57, subfam=nan- ['63-294', '333-565'] ATY63255.1
multiple domain ranges= -CazymeDomain in ATY66192.1, fam=GT2, subfam=nan- ['487-649', '627-867'] ATY66192.1

len PRE adding non-CAZyme= 445



len AFTER adding non-CAZyme= 9287
number of proteins in the input FASTA file of query protien sequences= 9287
protein count = 9287


From the printed out statements we can see that a `CazymeProteinPrediction` instance is created to represent the CAZyme/non-CAZyme and CAZyme domain prediction by each of the CAZyme prediction tools HMMER, Hotpep, DIAMOND and the consensus dbCAN results.

<a id="final_fun"></a>

## The final functions

The final task to make any last tweaks to the method for parsing the output from dbCAN, HMMER, Hotpep and DIAMOND, and combine these into a series of callable functions.

**Summary of the method:**  
The 'overview.txt' file from dbCAN contains all the desired data from dbCAN, except the EC numbers predicted by Hotpep. Therefore, to reduce the number of files that are required to be parsed, all required data is retrieved from the 'overview.txt' file, and this data is supplemented with the predicted EC numbers from the Hotpep 'Hotpep.out' output file.

The dbCAN result is the consensus result of the CAZyme prediction tools that are incorporated within dbCAN (HMMER, Gotpep, and DIAMOND). Specifically, the consensus result is any result that at least two of the CAZyme prediction tools agree upon. This applies to both the CAZyme/non-CAZyme prediction and the prediction of CAZyme domains (specifically the CAZy family and subfamily prediction for each CAZyme domain).

**Data retrieved from each CAZyme prediction tool**  

HMMER:
- CAZyme/non-CAZyme prediction
- CAZyme domain prediction (the CAZy family/subfamily prediction)
- Amino acid domain range prediction


Hotpep:
- CAZyme/non-CAZyme prediction
- CAZyme domain prediction (the CAZy family/subfamily prediction)
- EC number prediction per predicted CAZyme domain


DIAMOND:
- CAZyme/non-CAZyme prediction
- CAZyme domain prediction (the CAZy family/subfamily prediction)


dbCAN:
- CAZyme/non-CAZyme prediction
- CAZyme domain prediction (the CAZy family/subfamily prediction)

**Last little tweaks**  
When gathering the HMMER predictions, a check is performed to see if the same predicted CAZyme domain is listed more than once for a given protein. This is becusae multiple unique amino acid domain ranges can be predicted for the same CAZyme domain. This check has not been implemented in when retrieving the CAZyme domain predictions from Hotpep and DIAMOND. However, to improve the robustness of the method it is better to include this check (checking if a predicted CAZyme domain has been listed more than once for a given protein) when also parsing Hotpep and DIAMOND.

Moreover, a 'quality check' can be added to the retrieval of EC numbers from the 'Hotpep.out' file, specifically checking that when an EC number is retrieved, it is in the expected format and thus more likely to be an EC number.

In [53]:
def parse_dbcan_output(overview_path, hotpep_file, fasta_file):
    """Parse the data in the 'overview.txt' files from dbCAN.
    
    The 'overview.txt' file contains the CAZyme predicion data from
    HMMER, Hotpep and DIAMOND. A dbCAN consensus result is retrieved
    by finding predictions that at least to of the prediction tools
    agree upon.
    
    Each protein from the input FASTA file containing all query protein
    sequences parsed by dbCAN is represented by a CazymeProteinprediction
    instance. Each predicted CAZyme domains in each predicted CAZyme is
    represented by a CazymeDomain instance, which are sotred within their
    parent CAZyme's CazymeProteinPrediction instance. A CazymeProteinPrediction
    instance is created for each protein in the inpu FASTA file parsed
    by dbCAN and for each CAZyme prediction tool: dbCAN, HMMER, Hotpep and
    DIAMOND.
    
    :param overview_path: Path, path to dbCAN output overview.txt file
    :param hotpep_file: Path, path to Hotpep.out file
    :param fasta_file: Path, path to fasta file containing query protein sequences
          parsed by dbCAN
    
    Return 4 lists, each containing the CazymeProteinPrediction instances for
    HMMER, Hotpep, DIAMOND and dbCAN.
    """
    # open the overview.txt file
    with open(overview_path, "r") as ofh:
        overview_file = ofh.read().splitlines()
    
    # create an empty dictionary to store the CazymeProteinPrediction instances
    # {protein_accesion: CazymeProteinPrediction_instance}
    overview_dict = {}

    # skip the the first line in the file, becuase it contains the 'column' headings
    for line in tqdm(overview_file[1:], desc="Parsing overview.txt"):
        # separate the line content
        line = line.split("\t")
        
        protein_accession = line[0]
        
        # create a CazymeProteinPrediction instance each for HMMER, Hotpep and DIAMOND
        hmmer_prediction = get_hmmer_prediction(line[1], protein_accession)
        hotpep_prediction = get_hotpep_prediction(line[2], protein_accession)
        diamond_prediction = get_diamond_prediction(line[3], protein_accession)
        
        # get the dbCAN consensus result
        dbcan_prediction = get_dbcan_consensus(hmmer_prediction, hotpep_prediction, diamond_prediction, protein_accession, line[-1])
        
        # add the CazymeProteinPrediction instances to the overview_dict
        try:
            overview_dict[protein_accession]
            prediction_outputs = [["HMMER", hmmer_prediction], ["Hotpep", hotpep_prediction], ["DIAMOND", diamond_prediction], ["dbCAN", dbcan_prediction]]
            
            for prediction_tool in prediction_outputs:
                try:
                    overview_dict[protein_accession][prediction_tool[0]]
                    print(
                        f"WARNING -- Protein {protein_accession} already catalogued with {prediction_tool[0]}"
                    )
                except KeyError:  # raised if CazymeProteinPrediciton instance not stored for prediction_tool
                    overview_dict[protein_accession][prediction_tool[0]] = prediction_tool[1]
        
        except KeyError:  # raised if protein has not been added yet
            overview_dict[protein_accession] = {
                "HMMER": hmmer_prediction,
                "Hotpep": hotpep_prediction,
                "DIAMOND": diamond_prediction,
                "dbCAN": dbcan_prediction,
            }
    
    # get EC numbers predicted by Hotpep
    print("proteins in overview_dict pre EC=", len(list(overview_dict.values())))
    overview_dict = get_hotpep_ec_numbers(overview_dict, hotpep_file)
    print("proteins in overview_dictafter EC, pre non=", len(list(overview_dict.values())))
    
    # add proteins predicted to be non-CAZyme to the overview_dict
    overview_dict = get_non_cazymes(fasta_file, overview_dict)
    print("proteins in overview_dict after non=", len(list(overview_dict.values())))

    # now to retrieve a list each of CazymeProteinPredictions for HMMER, Hotpep, DIAMOND and dbCAN
    print("proteins in overview_dict=", len(list(overview_dict.values())))
    print("accessions=\n", list(overview_dict.keys()))
    hmmer_predictions, hotpep_predictions, diamond_predictions, dbcan_predictions = separate_predictions(overview_dict)
    
    return hmmer_predictions, hotpep_predictions, diamond_predictions, dbcan_predictions
        

def get_hmmer_prediction(hmmer_data, protein_accession):
    """Retrieve HMMER prediciton data from the dbCAN overview.txt file.
    
    :param hmmer_data: str, output data from HMMER written in the overview.txt file.
    :param protein_accession: str
    
    Return a CazymeProteinPrediction instance.
    """
    # check if the protein was classified as a non-CAZyme
    if hmmer_data == "-":
        cazyme_classification = 0  # non-CAZyme
        protein = CazymeProteinPrediction("HMMER", protein_accession, cazyme_classification)
        return protein
    
    # create a CazymeProteinPrediction instance to represent the CAZyme
    cazyme_classification = 1
    parent_cazyme = CazymeProteinPrediction("HMMER", protein_accession, cazyme_classification)

    # HMMER can predicted multiple CAZyme domaisn for a CAZyme
    # HMMER separates each of the predicted CAZyme domains by additiona symbols
    parent_cazyme.cazyme_domains = get_hmmer_cazyme_domains(hmmer_data.split("+"), protein_accession)
    
    return parent_cazyme

    
def get_hmmer_cazyme_domains(predicted_domains, protein_accession):
    """Retrieve the CAZyme domains predicted by HMMER.
    
    :param predicted_domains: list of strings, each string containing a CAZyme domain prediction
    :param protein_accession: str
    
    Return list of CazymeDomain instances. Each instance represents a unique CAZyme domain.
    """
    cazyme_domains = {}  # {"cazyfam-cazysubfam": CazymeDomain_instance}

    for domain in predicted_domains:
        # separate the predicted CAZy family from the domain range
        domain = domain.split("(")
        
        # get the predicted CAZy family(subfamily)
        domain_name = domain[0]  # CAZy (sub)family
        cazy_family, cazy_subfamily = get_hmmer_cazy_family(domain_name)
        
        if cazy_family is None:  # unexpected data format found by get_hmmer_cazy_family()
            continue
        
        # get the predicted amino acid domain range
        domain_range = domain[1][:-1]  # drop the terminal ")"
        # check it matches the expected format and thus is the predicted domain range
        try:
            re.match(r"\d+?-\d+", domain_range).group()
        except AttributeError:  # raised if not correct format
            print(
                f"WARNING -- UNKNOWN DATA TYPE OF {domain_range} FOR {protein_accession}\n"
                f"Not adding domain range for the domain {cazy_family}-{cazy_subfamily}"
            )
            domain_range = np.nan

        # create identifier for the CAZyme domain to check if it has already been catalogued
        domain_identifier = f"{cazy_family}-{cazy_subfamily}"
    
        # check if the CAZyme domain has already been cataglouged
        try:
            existing_domain = cazyme_domains[domain_identifier]

            # CAZyme domain already catalogued, check if the retrieved domain range is the same
            if domain_range not in existing_domain.domain_range:
                # append the new domain_range
                existing_domain.domain_range.append(domain_range)
                print("multiple domain ranges=", existing_domain, existing_domain.domain_range, protein_accession)

        except KeyError:  # raised if the CAZyme domain has not yet been catalogued
            cazyme_domains[domain_identifier] = CazymeDomain(
                prediction_tool="HMMER",
                protein_accession=protein_accession,
                cazy_family=cazy_family,
                cazy_subfamily=cazy_subfamily,
                domain_range=[domain_range],
            )

    return list(cazyme_domains.values())


def get_hmmer_cazy_family(domain_name):
    """Retrieve the name of the CAZyme domain, this is the predicted CAZy family and subfamily.
    
    If a subfamily is not predicted, the CAZy subfamily is set as a null value.
    If the data formate does not match the expected format, the CAZy family will
    be set by None, which will be used to not add the current working domain to
    the parent CazymeProteinPrediction instance.
    
    :param domain_name: str, contains the CAZy (sub)family
    
    Return two strings, the CAZy family and the CAZy subfamily.
    """
    # check if unusal CAZy family format or subfamily was given
    if domain_name.find("_") != -1:
        try:
            re.match(r"\D{2,3}\d+?_\D", domain_name).group()  # check unusal CAZy family formating
            cazy_family = domain_name[:domain_name.find("_")]
            cazy_subfamily = np.nan

        except AttributeError:  # raised if not an usual CAZy family format
            try:
                # check if its a CAZy subfamily
                re.match(r"\D{2,3}\d+?_\d+", domain_name).group()
                cazy_family = domain_name[:domain_name.find("_")]
                cazy_subfamily = np.nan

            except AttributeError:  # raised if it doesn't match the expected CAZy subfamily format
                print(
                    f"WARNING -- UNKNOWN DATA TYPE OF {domain_name} FOR {protein_accession}\n"
                    "Not adding as a domain."
                )
                return None, None

    else:
        # potentially have a CAZy family
        try:
            re.match(r"\D{2,3}\d+", domain_name).group()
            cazy_family = domain_name
            cazy_subfamily = np.nan

        except AttributeError:  # raised if doesn't match expected CAZy family
            print(
                f"WARNING -- UNKNOWN DATA TYPE OF {domain_name} FOR {protein_accession}\n"
                "Not adding as a domain."
            )
            return None, None
    
    return cazy_family, cazy_subfamily


def get_hotpep_prediction(hotpep_data, protein_accession):
    """Retrieve Hotpep prediciton data from the dbCAN overview.txt file.
    
    :param hmmer_data: str, output data from Hotpep written in the overview.txt file.
    :param protein_accession: str
    
    Return a CazymeProteinPrediction instance.
    """
    # check if the protein was classified as a non-CAZyme
    if hotpep_data == "-":
        cazyme_classification = 0  # non-CAZyme
        protein = CazymeProteinPrediction("Hotpep", protein_accession, cazyme_classification)
        return protein
    
    # create a CazymeProteinPrediction instance to represent the CAZyme
    cazyme_classification = 1
    parent_cazyme = CazymeProteinPrediction("Hotpep", protein_accession, cazyme_classification)

    # Hotpep can predicted multiple CAZyme domaisn for a CAZyme
    # Hotpep separates each of the predicted CAZyme domains by additiona symbols
    parent_cazyme.cazyme_domains = get_hotpep_cazyme_domains(hotpep_data.split("+"), protein_accession)
    
    return parent_cazyme


def get_hotpep_cazyme_domains(predicted_domains, protein_accession):
    """Retrieve the CAZyme domains predicted by Hotpep.
    
    :param predicted_domains: list of strings, each string containing a CAZyme domain prediction.
    
    Return list of CazymeDomain instances. Each instance represents a unique CAZyme domain.
    """
    cazyme_domains = {}  # {"cazyfam-cazysubfam": CazymeDomain}
    
    for domain in predicted_domains:
        # remove k-mer group number
        domain = domain.split("(")[0]
        
        # check if a CAZy subfamily was predicted
        if domain.find("_") != -1:  # found a CAZy subfamily
            try:
                # check if its a CAZy subfamily
                re.match(r"\D{2,3}\d+?_\d+", domain).group()
                cazy_family = domain[:domain.find("_")]
                cazy_subfamily = np.nan

            except AttributeError:  # raised if it doesn't match the expected CAZy subfamily format
                print(
                    f"WARNING -- UNKNOWN DATA TYPE OF {domain} FOR {protein_accession}\n"
                    "Not adding as a domain."
                )
                continue
        
        else:  # found a CAZy family
            try:  # check it is a CAZy family
                re.match(r"\D{2,3}\d+", domain).group()
                cazy_family = domain
                cazy_subfamily = np.nan

            except AttributeError:  # raised if doesn't match expected CAZy family
                print(
                    f"WARNING -- UNKNOWN DATA TYPE OF {domain} FOR {protein_accession}\n"
                    "Not adding as a domain."
                )
                continue
        
        new_domain = CazymeDomain("Hotpep", protein_accession, cazy_family, cazy_subfamily)
        
        try:
            cazyme_domains[f"{cazy_family}-{cazy_subfamily}"]
            print(
                f"WARNING -- DUPLICATE CAZYME DOMAIN PREDICTIONS OF CAZy family = {cazy_family},\n"
                f" CAZy subfamily = {cazy_subfamily}, in protein {protein_accession}.\n"
                "Only one CazymeDomain instance being added to the respective CazymeProteinPrediction instance."
            )
        except KeyError:
            cazyme_domains[f"{cazy_family}-{cazy_subfamily}"] = new_domain
    
    return list(cazyme_domains.values())
        

def get_hotpep_ec_numbers(overview_dict, hotpep_file):
    """Retrieve EC numbers from the Hotpep.out file, and add them to the CAZyme domains.
    
    :param overview_dict: dict, keyed by protein accession, valued by dictionary of 
                            predicion_tool:CazymeDomain
    :param hotpep_file: Path, path to Hotpep.out file
    
    Return overview_dict
    """
    with open (hotpep_file, "r") as fh:
        hotpep_file_data = fh.read().splitlines()
    
    for line in tqdm(hotpep_file_data[1:], desc="Parsing Hotpep.out"):  # skip the first line containing headings
        # separate out the data
        line = line.split("\t")
        
        protein_accession = line[2].strip()
        
        # retrieve the EC numbers
        ec_numbers = line[-1]
        ec_numbers = ec_numbers.split(",")

        # strip any remaining white space becuase the separating comma
        # is sometimes followed by a space, and remove the kmer cluster number
        index = 0
        for index in range(len(ec_numbers)):
            ec_numbers[index] = ec_numbers[index].strip()
            # remove k-mer group number (EC:kmer_group#)
            ec_numbers[index] = ec_numbers[index].split(":")[0]
        
            if ec_numbers[index] == "NA":
                ec_numbers[index] = np.nan
                continue

            try:
                re.match(r"\d+?\.(\d+?|-)\.(\d+?|-)\.(\d+?|-)", ec_numbers[index])  # missing digits listed as '-'

            except AttributeError:  # raised if not in the expected EC number format
                print(
                    f"WARNING -- unexpected data format of '{ec_numbers[index]}' for protein "
                    f"{protein_accession}.\n"
                    "Not adding this data to the CAZyme's CazymeProteinPrediction instance."
                )
                ec_numbers.remove(ec_numbers[index])
        
        # get CAZy family and subfamily of the current working domain
        cazy_family = line[0]
        if cazy_family.find("_") != -1:
            cazy_subfamily = cazy_family
            cazy_family = cazy_family[:cazy_family.find("_")]
        else:
            cazy_subfamily = np.nan

        domain_identifer = [cazy_family, cazy_subfamily]
        
        parent_cazyme = overview_dict[protein_accession]["Hotpep"]
        parent_domains = parent_cazyme.cazyme_domains
        
        for domain in parent_domains:
            if domain_identifer == [domain.cazy_family, domain.cazy_subfamily]:
                domain.ec_numbers = ec_numbers
        
        parent_cazyme.cazyme_domains = parent_domains
        overview_dict[protein_accession]["Hotpep"] = parent_cazyme
        
    return overview_dict


def get_diamond_prediction(diamond_data, protein_accession):
    """Retrieve DIAMOND prediction data from dbCAN overview.txt file.
    
    :param diamond_data, str, output data from DIAMOND written in the overview.txt
    :param protein_accession, str
    
    Return a CazymeProteinPrediction instance.
    """
    # check if the protein was classified as a non-CAZyme
    if diamond_data == "-":
        cazyme_classification = 0  # non-CAZyme
        protein = CazymeProteinPrediction("DIAMOND", protein_accession, cazyme_classification)
        return protein
    
    # create a CazymeProteinPrediction instance to represent the CAZyme
    cazyme_classification = 1
    parent_cazyme = CazymeProteinPrediction("DIAMOND", protein_accession, cazyme_classification)
    
    # DIAMOND can predict multiple CAZyme domains per CAZyme, each domain is separated by '+'
    parent_cazyme.cazyme_domains = get_diamond_predicted_domains(diamond_data.split("+"), protein_accession)
 
    return parent_cazyme


def get_diamond_predicted_domains(predicted_domains, protein_accession):
    """Retrieve the CAZyme domains predicted by DIAMOND, from the overview.txt file.
    
    :param predicted_domains: list of strings, each string contains a unique CAZyme domain prediction
    :param protein_accession: str
    
    Return list of CazymeDomain instances, one instance per unique CAZyme domain.
    """
    cazyme_domains = {}  # store CazymeDomain instances, keyed by CAZyme domain "fam-subfam"
    
    for predicted_domain in predicted_domains:
        # check if a subfamily was predicted
        try:
            re.match(r"\D{2,3}\d+?_\d+", predicted_domain).group()
            cazy_family = predicted_domain[:predicted_domain.find("_")]
            cazy_subfamily = predicted_domain
        
        except AttributeError:  # raised if predicted_domain is not a CAZy subfamily
            try:
                # check if a CAZy family was predicted
                re.match(r"\D{2,3}\d+", predicted_domain).group()
                cazy_family = predicted_domain
                cazy_subfamily = np.nan
            
            except AttributeError:  # raised if not a CAZy family
                print(
                    f"WARNING -- Unexpected data formate for '{predicted_domain}' for\n"
                    "DIAMOND in the overview.txt file. Not adding domain to the CAZyme."
                )
            continue
        
        new_domain = CazymeDomain("DIAMOND", protein_accession, cazy_family, cazy_subfamily)
        
        # check if the domain has been parsed before
        try:
            cazyme_domains[f"{cazy_family}-{cazy_subfamily}"]
            print(
                f"WARNING -- duplicate CAZyme domains predicted for protein {protein_accession} "
                " by DIAMOND.\n"
                f"CAZyme domain (CAZy family = {cazy_family}, CAZy subfamily = {cazy_subfamily}),\n"
                "added to the protein once as a single CazymeDomain instance"
            )
            continue
        
        except KeyError:
            cazyme_domains[f"{cazy_family}-{cazy_subfamily}"] = new_domain

    
    return list(cazyme_domains.values())


def get_dbcan_consensus(hmmer_prediction, hotpep_prediction, diamond_prediction, protein_prediction, no_of_tools):
    """Retrieve the consensus CAZyme and CAZyme domain prediction for dbCAN.
    
    Consensus is defined as a result that at least two of the tools (HMMER, Hotpep and DIAMOND)
    agree upon.
    
    :param hmmer_prediction: CazymeProteinPrediction instance containing the HMMER prediction
    :param hotpep_prediction: CazymeProteinPrediction instance containing the Hotpep prediction
    :param diamond_prediction: CazymeProteinPrediction instance containing the DIAMOND prediction
    :param protein_accession: str
    :param no_of_tools: str, data from the '#ofTools' column from the overview.txt file, it states
        the number of tools that list the protein as a CAZyme
    
    Return CazymeProteinPrediction instance to represent the dbCAN consensus prediction
    """
    # first check if the consensus was that the protein was a non-CAZyme
    if no_of_tools == "1":
        # non-CAZyme
        cazyme_classification = 0  # non-CAZyme
        protein = CazymeProteinPrediction("dbCAN", protein_accession, cazyme_classification)
        return protein
    
    # get the consensus CAZyme domain predicitons
    
    # first get each of the predicted CAZyme domains for each tool
    hmmer_domains = get_dbcan_cazyme_domains(hmmer_prediction)
    hotpep_domains = get_dbcan_cazyme_domains(hotpep_prediction)
    diamond_domains = get_dbcan_cazyme_domains(diamond_prediction)

    # get the predicted CAZyme domains that are predicted by at two tools
    hmmer_hotpep_consensus = list(set(hmmer_domains) & set(hotpep_domains))
    hotpep_diamond_consensus = list(set(hotpep_domains) & set(diamond_domains))
    hmmer_diamond_consensus = list(set(hmmer_domains) & set(diamond_domains))

    # get the CAZyme domains predicted by all prediction tools
    all_consensus = list(set(hmmer_domains) & set(hotpep_domains) & set(diamond_domains))

    # combine all consensus results
    combined_consensus = (
        hmmer_hotpep_consensus + hotpep_diamond_consensus + hmmer_diamond_consensus + all_consensus
    )
    # remove duplicate entries
    combined_consensus = list(set(combined_consensus))
    
    # create a CazymeProteinPrediction instance to represent the CAZyme
    cazyme_classification = 1  # CAZyme
    parent_cazyme = CazymeProteinPrediction("dbCAN", protein_accession, cazyme_classification)
    
    # create a CazymeDomain instance for each domain
    cazyme_domains = []  # store CazymeDomain instances
    for domain in combined_consensus:
        domain = domain.split("**")
        cazy_family = domain[0]

        if domain[1] == "nan":
            cazy_subfamily = np.nan
        else:
            cazy_subfamily = domain[1]

        new_domain = CazymeDomain("dbCAN", protein_accession, cazy_family, cazy_subfamily)

        cazyme_domains.append(new_domain)
    
    # add predicted CAZyme domains to the instance representing the CAZyme
    parent_cazyme.cazyme_domains = cazyme_domains
    
    return parent_cazyme
    
    
def get_dbcan_cazyme_domains(cazyme_prediction):
    """Retrieve the predicted CAZyme domains, representing each domains as a string 'fam_subfam'.
    
    :param cazyme_prediction: CazymeProteinPrediction instance.
    
    Return a list of strings with each string representing a single predicted CAZyme domain.
    """
    cazyme_domain_instances = cazyme_prediction.cazyme_domains
    
    cazyme_domains = []  # store list representing CAZyme domains [fam(str), subfam(str)]
    
    # if a prediction tool predicts a protein is a non_CAZyme, non_cazymes == np.nan (a float)
    if type(cazyme_domain_instances) is not float:
        for domain in cazyme_domain_instances:
            cazy_family = domain.cazy_family
            cazy_subfamily = domain.cazy_subfamily
            # separate the family and subfamily '**' becuase these do not appear in the either
            cazyme_domains.append(f"{cazy_family}**{cazy_subfamily}")
    
    return cazyme_domains


def get_non_cazymes(fasta_path, overview_dict):
    """Add proteins that dbCAN identified as non-CAZymes to the collection of CazymeProteinPrediction instances.
    
    :param fasta_path: Path, FASTA file used as input for CUPP.
    :param overview_dict: dict, key=protein_accession, value=dict valued by prediction tool,
          valued by CazymeProteinPrediction intance
    
    Return a dictionary keyed by protein accessions and valued by dictionary of tool:CazymeProteinPrediction.
    """
    # open the FASTA path
    with open(fasta_path) as fh:
        fasta = fh.read().splitlines()

    for line in tqdm(fasta, desc="Adding non-CAZymes"):
        if line.startswith(">"):
            protein_accession = line[1:].strip()
            
            # check if the protein has been listed as a CAZyme by CUPP
            try:
                overview_dict[protein_accession]
            except KeyError:
                # raised if protein not in cupp_predictions, inferring proten was not labelled as CAZyme
                cazyme_classification = 0
                overview_dict[protein_accession] = {}
                for prediction_tool in ["HMMER", "Hotpep", "DIAMOND", "dbCAN"]:
                    overview_dict[protein_accession][prediction_tool] = CazymeProteinPrediction(
                        prediction_tool,
                        protein_accession,
                        cazyme_classification,
                    )

    return overview_dict


def separate_predictions(overview_dict):
    """Create separate lists of CazymeProteinPrediction intances for each prediction tool.
    
    :param overview_dict: dict containing all CazymeProteinPrediciton instances
        {protein_accession: {tool: CazymeProteinPrediction instance}}
    
    Return 4 lists of CazymeProteinPrediction instances    
    """
    hmmer_predictions = []
    hotpep_predictions = []
    diamond_predictions = []
    dbcan_predictions = []
    for protein_accession in list(overview_dict.keys()):
        try:
            hmmer_predictions.append(overview_dict[protein_accession]["HMMER"])
        except KeyError:
            print(
                f"Could not find HMMER prediction for {protein_accession}"
            )

        try:
            hotpep_predictions.append(overview_dict[protein_accession]["Hotpep"])
        except KeyError:
            print(
                f"Could not find HMMER prediction for {protein_accession}"
            )

        try:
            diamond_predictions.append(overview_dict[protein_accession]["DIAMOND"])
        except KeyError:
            print(
                f"Could not find HMMER prediction for {protein_accession}"
            )

        try:
            dbcan_predictions.append(overview_dict[protein_accession]["dbCAN"])
        except KeyError:
            print(
                f"Could not find HMMER prediction for {protein_accession}"
            )
    
    return hmmer_predictions, hotpep_predictions, diamond_predictions, dbcan_predictions
        


overview_path = "overview.txt"
hotpep_file = "Hotpep.out"
fasta_file = "genbank_proteins_txid73501_GCA_008080495_1.fasta"
hmmer_predictions, hotpep_predictions, diamond_predictions, dbcan_predictions = parse_dbcan_output(overview_path, hotpep_file, fasta_file)
print("hmmer=", len(hmmer_predictions))
print("hotpep=", len(hotpep_predictions))
print("diamond=", len(diamond_predictions))
print("dbcan=", len(dbcan_predictions))

multiple domain ranges= -CazymeDomain in ATY60367.1, fam=CBM66, subfam=nan- ['219-369', '388-547'] ATY60367.1
multiple domain ranges= -CazymeDomain in ATY60416.1, fam=GT2, subfam=nan- ['230-398', '375-584'] ATY60416.1
multiple domain ranges= -CazymeDomain in ATY63255.1, fam=GT57, subfam=nan- ['63-294', '333-565'] ATY63255.1
multiple domain ranges= -CazymeDomain in ATY66192.1, fam=GT2, subfam=nan- ['487-649', '627-867'] ATY66192.1

proteins in overview_dict pre EC= 445



proteins in overview_dictafter EC, pre non= 445



proteins in overview_dict after non= 9732
proteins in overview_dict= 9732
accessions=
 ['ATY58323.1', 'ATY58330.1', 'ATY58395.1', 'ATY58396.1', 'ATY58475.1', 'ATY58478.1', 'ATY58486.1', 'ATY58494.1', 'ATY58512.1', 'ATY58528.1', 'ATY58531.1', 'ATY58532.1', 'ATY58571.1', 'ATY58579.1', 'ATY58636.1', 'ATY58637.1', 'ATY58653.1', 'ATY58674.1', 'ATY58687.1', 'ATY58705.1', 'ATY58803.1', 'ATY58848.1', 'ATY58880.1', 'ATY58895.1', 'ATY58902.1', 'ATY58910.1', 'ATY58915.1', 'ATY58949.1', 'ATY58965.1', 'ATY59023.1', 'ATY59031.1', 'ATY59065.1', 'ATY59093.1', 'ATY59185.1', 'ATY59188.1', 'ATY59194.1', 'ATY59202.1', 'ATY59250.1', 'ATY59363.1', 'ATY59524.1', 'ATY59596.1', 'ATY59638.1', 'ATY59663.1', 'ATY59671.1', 'ATY59673.1', 'ATY59688.1', 'ATY59691.1', 'ATY59698.1', 'ATY59714.1', 'ATY59728.1', 'ATY59756.1', 'ATY59775.1', 'ATY59781.1', 'ATY59860.1', 'ATY59890.1', 'ATY59904.1', 'ATY59918.1', 'ATY59933.1', 'ATY59950.1', 'ATY59985.1', 'ATY60044.1', 'ATY60072.1', 'ATY60094.1', 'ATY60133.1', 'ATY60146.1', '

Here we can see that all proteins from the input FASTA file are parsed, creating a single `CazymeProteinPrediction` instance per protein. Additionally, we have a robust method for checking if duplicates are found, dealing with those, and also ensuring the returned data matches the expected format.